# Data Generation Workbench

## Purpose

This notebook provides an interactive workbench for generating synthetic AM-QADF datasets. Use widgets to configure data generation parameters, preview generated data, and populate MongoDB collections - all without writing code!

## Learning Objectives

By the end of this notebook, you will:
- ✅ Configure STL processing and hatching generation parameters
- ✅ Generate laser parameters, ISPM data, and CT scan data interactively
- ✅ Preview generated data before committing to MongoDB
- ✅ Populate MongoDB collections with generated datasets
- ✅ Understand the relationship between parameters and generated data
- ✅ Create reproducible datasets using random seeds

## Estimated Duration

60-90 minutes

---

## Overview

The Data Generation Workbench enables interactive creation of synthetic AM-QADF datasets:

- 📐 **STL Processing**: Load and process 3D STL models
- 🛤️ **Hatching Generation**: Generate layer-by-layer scan paths
- ⚡ **Laser Parameters**: Create realistic laser process parameters
- 🌡️ **ISPM Data**: Generate in-situ process monitoring data
- 🔬 **CT Scans**: Create computed tomography scan data
- 💾 **MongoDB Population**: Store generated data in the warehouse

---

## Workflow

**Quick Start (Simplest Way):**
1. **Run Setup** (Cell 1): Import libraries and connect to MongoDB
2. **Select STL Model**: Choose an STL file from the dropdown
3. **Click Generate**: All data types (hatching, laser, ISPM, CT scan) will be generated with default parameters

**Advanced Usage:**
- **Configure Parameters**: Adjust parameters in each accordion section before generating
- **Select Collections**: Choose which MongoDB collections to populate (defaults to all)
- **Preview** (optional): Click "Preview" to see what will be generated without saving
- **Load STL** (optional): Click "Load STL" to preview the model and extract metadata

**Note**: The "Generate" button is enabled as soon as you select an STL file. It will automatically load the STL and generate all data types with the current widget parameters (defaults if unchanged).

---

Use the interactive widgets below to explore data generation - all parameters are adjustable in real-time!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML, interactive, interact, fixed, Textarea
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List, Callable
import json
import time
import uuid

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    from src.infrastructure.config import MongoDBConfig
    INFRASTRUCTURE_AVAILABLE = True
except Exception as e:
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}")

# Try to import generation modules
GENERATION_AVAILABLE = False
try:
    from generation.process.stl_processor import STLProcessor
    from generation.process.hatching_generator import HatchingGenerator, HatchingConfig
    from generation.sensors.laser_parameter_generator import LaserParameterGenerator
    from generation.sensors.ispm_generator import ISPMGenerator
    from generation.sensors.ct_scan_generator import CTScanGenerator
    GENERATION_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Generation modules not available: {e}")
    GENERATION_AVAILABLE = False

# Initialize MongoDB connection
mongo_client = None
if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        if mongo_client and mongo_client.is_connected():
            print("✅ MongoDB connection established")
        else:
            print("⚠️ MongoDB connection failed")
    except Exception as e:
        print(f"⚠️ MongoDB connection error: {type(e).__name__}: {e}")
        print("   Using demo mode with synthetic data")
else:
    print("⚠️ Using demo mode - infrastructure layer unavailable")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ MongoDB connection established
✅ Setup complete!


## Interactive Data Generation Interface

Use the widgets below to configure data generation parameters, preview generated data, and populate MongoDB collections. The interface is organized into configuration panels, visualization areas, and status displays.

**Run the cell below to create the complete interface.**


In [2]:
# ============================================================================
# COMPLETE WIDGET INTERFACE DEFINITION
# ============================================================================
# This cell defines all widgets and assembles the complete interface
# ============================================================================

# Global state
generated_data = {
    'stl_model': None,
    'hatching_data': None,
    'laser_params': None,
    'ispm_data': None,
    'ct_scan': None,
    'model_id': None
}

# ============================================
# Section 1: Top Panel - Generation Mode and Actions
# ============================================

# Generation mode selector
generation_mode = RadioButtons(
    options=[
        ('Single Model', 'single'),
        ('Batch', 'batch'),
        ('Custom Scenario', 'custom'),
        ('Quick Demo', 'demo')
    ],
    value='single',
    description='Mode:',
    layout=Layout(width='200px')
)

# STL model selector
stl_models_dir = project_root / 'generation' / 'models'
available_stl_files = []
if stl_models_dir.exists():
    available_stl_files = sorted([f.name for f in stl_models_dir.glob('*.stl')])

# Create options with "Choose a model" as first option
stl_options = ['Choose a model']
if available_stl_files:
    stl_options.extend(available_stl_files)
else:
    stl_options = ['No STL files found']

stl_selector = Dropdown(
    options=stl_options,
    value='Choose a model',  # Default to placeholder
    description='STL Model:',
    layout=Layout(width='300px'),
    disabled=len(available_stl_files) == 0
)

# Action buttons
generate_button = Button(
    description='Generate',
    button_style='success',
    icon='play',
    layout=Layout(width='120px'),
    disabled=True,  # Disabled until ready
    tooltip='Load STL and configure parameters first'
)

preview_button = Button(
    description='Preview',
    button_style='info',
    icon='eye',
    layout=Layout(width='120px'),
    tooltip='Preview generation configuration'
)

clear_preview_button = Button(
    description='Clear Preview',
    button_style='warning',
    icon='trash',
    layout=Layout(width='140px')
)

save_config_button = Button(
    description='Save Config',
    button_style='',
    icon='save',
    layout=Layout(width='140px')
)

# Status label
generate_status_label = WidgetHTML(
    value='<i style="color:gray; font-size:0.9em;">⚠️ Choose a model to enable Generate</i>',
    layout=Layout(margin='0 10px')
)

# Top panel layout
top_panel = VBox([
    HBox([
        generation_mode,
        stl_selector,
        generate_button,
        preview_button,
        clear_preview_button,
        save_config_button
    ], layout=Layout(
        justify_content='flex-start',
        align_items='center',
        padding='5px'
    )),
    HBox([generate_status_label], layout=Layout(padding='5px'))
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

# ============================================
# Section 2: Left Panel - Configuration Accordion
# ============================================

# --- STL Processing Configuration ---
stl_load_button = Button(
    description='Load STL',
    button_style='primary',
    icon='upload',
    layout=Layout(width='150px')
)

stl_preview_toggle = Checkbox(
    value=True,
    description='Show 3D Preview',
    layout=Layout(width='150px')
)

stl_extract_metadata = Checkbox(
    value=True,
    description='Extract Metadata',
    layout=Layout(width='150px')
)

stl_calc_bbox = Checkbox(
    value=True,
    description='Calculate BBox',
    layout=Layout(width='150px')
)

stl_calc_volume = Checkbox(
    value=True,
    description='Calculate Volume',
    layout=Layout(width='150px')
)

stl_metadata_display = Textarea(
    value='',
    description='Metadata:',
    layout=Layout(width='100%', height='150px'),
    disabled=True
)

stl_config_panel = VBox([
    WidgetHTML("<b>STL Processing Configuration</b>"),
    HBox([stl_load_button, stl_preview_toggle]),
    HBox([stl_extract_metadata, stl_calc_bbox, stl_calc_volume]),
    stl_metadata_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- Hatching Generation Configuration ---
layer_thickness = FloatSlider(
    value=0.05,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Layer Thickness (mm):',
    layout=Layout(width='100%')
)

hatch_spacing = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.01,
    description='Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_angle = FloatSlider(
    value=45.0,
    min=0.0,
    max=180.0,
    step=5.0,
    description='Hatch Angle (deg):',
    layout=Layout(width='100%')
)

contour_offset = FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Contour Offset (mm):',
    layout=Layout(width='100%')
)

max_layers = IntSlider(
    value=50,
    min=1,
    max=1000,
    step=1,
    description='Max Layers:',
    layout=Layout(width='100%')
)

sampling_spacing = FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Sampling Spacing (mm):',
    layout=Layout(width='100%')
)

laser_beam_width = FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Laser Beam Width (mm):',
    layout=Layout(width='100%')
)

hatch_pattern = RadioButtons(
    options=[('Standard', 'standard'), ('Island', 'island'), ('Contour', 'contour')],
    value='standard',
    description='Pattern:',
    layout=Layout(width='100%')
)

preview_hatching_button = Button(
    description='Preview Hatching',
    button_style='info',
    icon='eye',
    layout=Layout(width='180px')
)

hatching_stats_display = WidgetHTML(value='<i>No hatching generated yet</i>')

hatching_config_panel = VBox([
    WidgetHTML("<b>Hatching Generation Configuration</b>"),
    layer_thickness,
    hatch_spacing,
    hatch_angle,
    contour_offset,
    max_layers,
    sampling_spacing,
    laser_beam_width,
    hatch_pattern,
    HBox([preview_hatching_button]),
    hatching_stats_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- Laser Parameters Configuration ---
power_mean = FloatSlider(
    value=250.0,
    min=50.0,
    max=500.0,
    step=5.0,
    description='Mean Power (W):',
    layout=Layout(width='100%')
)

power_std = FloatSlider(
    value=25.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Std Power (W):',
    layout=Layout(width='100%')
)

power_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=400.0,
    step=5.0,
    description='Min Power (W):',
    layout=Layout(width='100%')
)

power_max = FloatSlider(
    value=400.0,
    min=100.0,
    max=500.0,
    step=5.0,
    description='Max Power (W):',
    layout=Layout(width='100%')
)

speed_mean = FloatSlider(
    value=500.0,
    min=100.0,
    max=3000.0,
    step=50.0,
    description='Mean Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_std = FloatSlider(
    value=50.0,
    min=0.0,
    max=200.0,
    step=5.0,
    description='Std Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=2000.0,
    step=50.0,
    description='Min Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_max = FloatSlider(
    value=2000.0,
    min=500.0,
    max=3000.0,
    step=50.0,
    description='Max Speed (mm/s):',
    layout=Layout(width='100%')
)

hatch_spacing_mean = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.01,
    description='Mean Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_spacing_std = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Std Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

contour_power_mult = FloatSlider(
    value=1.2,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Power Mult:',
    layout=Layout(width='100%')
)

contour_speed_mult = FloatSlider(
    value=0.8,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Speed Mult:',
    layout=Layout(width='100%')
)

support_power_mult = FloatSlider(
    value=0.7,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Power Mult:',
    layout=Layout(width='100%')
)

support_speed_mult = FloatSlider(
    value=1.2,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Speed Mult:',
    layout=Layout(width='100%')
)

laser_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_laser_button = Button(
    description='Preview Laser Params',
    button_style='info',
    icon='zap',
    layout=Layout(width='180px')
)

laser_config_panel = VBox([
    WidgetHTML("<b>Laser Parameters Configuration</b>"),
    WidgetHTML("<i>Power Configuration</i>"),
    power_mean, power_std, power_min, power_max,
    WidgetHTML("<i>Speed Configuration</i>"),
    speed_mean, speed_std, speed_min, speed_max,
    WidgetHTML("<i>Hatch Spacing</i>"),
    hatch_spacing_mean, hatch_spacing_std,
    WidgetHTML("<i>Region Multipliers</i>"),
    contour_power_mult, contour_speed_mult,
    support_power_mult, support_speed_mult,
    HBox([laser_random_seed, preview_laser_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- ISPM Data Configuration ---
base_temperature = FloatSlider(
    value=1700.0,
    min=1000.0,
    max=2500.0,
    step=10.0,
    description='Base Temperature (K):',
    layout=Layout(width='100%')
)

temperature_variation = FloatSlider(
    value=100.0,
    min=0.0,
    max=500.0,
    step=5.0,
    description='Temperature Variation (K):',
    layout=Layout(width='100%')
)

melt_pool_width_mean = FloatSlider(
    value=0.5,
    min=0.1,
    max=2.0,
    step=0.05,
    description='Melt Pool Width Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_width_std = FloatSlider(
    value=0.1,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Width Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_mean = FloatSlider(
    value=1.0,
    min=0.2,
    max=5.0,
    step=0.05,
    description='Length Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_std = FloatSlider(
    value=0.15,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Length Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_mean = FloatSlider(
    value=0.3,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Depth Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_std = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Depth Std (mm):',
    layout=Layout(width='100%')
)

cooling_rate_mean = FloatSlider(
    value=100.0,
    min=10.0,
    max=500.0,
    step=5.0,
    description='Cooling Rate Mean (K/s):',
    layout=Layout(width='100%')
)

cooling_rate_std = FloatSlider(
    value=20.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Cooling Rate Std (K/s):',
    layout=Layout(width='100%')
)

temp_gradient_mean = FloatSlider(
    value=50.0,
    min=10.0,
    max=200.0,
    step=5.0,
    description='Temp Gradient Mean (K/mm):',
    layout=Layout(width='100%')
)

temp_gradient_std = FloatSlider(
    value=10.0,
    min=0.0,
    max=50.0,
    step=1.0,
    description='Temp Gradient Std (K/mm):',
    layout=Layout(width='100%')
)

sampling_rate = FloatSlider(
    value=1000.0,
    min=100.0,
    max=10000.0,
    step=100.0,
    description='Sampling Rate (Hz):',
    layout=Layout(width='100%')
)

points_per_layer = IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Points per Layer:',
    layout=Layout(width='100%')
)

ispm_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_thermal_button = Button(
    description='Preview ISPM Data',
    button_style='info',
    icon='thermometer',
    layout=Layout(width='180px')
)

ispm_config_panel = VBox([
    WidgetHTML("<b>ISPM Data Configuration</b>"),
    WidgetHTML("<i>Temperature Configuration</i>"),
    base_temperature, temperature_variation,
    WidgetHTML("<i>Melt Pool Size</i>"),
    melt_pool_width_mean, melt_pool_width_std,
    melt_pool_length_mean, melt_pool_length_std,
    melt_pool_depth_mean, melt_pool_depth_std,
    WidgetHTML("<i>Thermal Parameters</i>"),
    cooling_rate_mean, cooling_rate_std,
    temp_gradient_mean, temp_gradient_std,
    WidgetHTML("<i>Sampling</i>"),
    sampling_rate, points_per_layer,
    HBox([ispm_random_seed, preview_thermal_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- CT Scan Configuration ---
ct_x_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='X Dimension:',
    layout=Layout(width='100%')
)

ct_y_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Y Dimension:',
    layout=Layout(width='100%')
)

ct_z_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Z Dimension:',
    layout=Layout(width='100%')
)

ct_x_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='X Spacing (mm):',
    layout=Layout(width='100%')
)

ct_y_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Y Spacing (mm):',
    layout=Layout(width='100%')
)

ct_z_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Z Spacing (mm):',
    layout=Layout(width='100%')
)

base_density = FloatSlider(
    value=8.0,
    min=1.0,
    max=20.0,
    step=0.1,
    description='Base Density (g/cm³):',
    layout=Layout(width='100%')
)

density_variation = FloatSlider(
    value=0.2,
    min=0.0,
    max=2.0,
    step=0.05,
    description='Density Variation:',
    layout=Layout(width='100%')
)

base_porosity = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Base Porosity:',
    layout=Layout(width='100%')
)

porosity_variation = FloatSlider(
    value=0.005,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Porosity Variation:',
    layout=Layout(width='100%')
)

defect_probability = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Defect Probability:',
    layout=Layout(width='100%')
)

defect_size_min = IntSlider(
    value=2,
    min=1,
    max=50,
    step=1,
    description='Defect Size Min (voxels):',
    layout=Layout(width='100%')
)

defect_size_max = IntSlider(
    value=10,
    min=2,
    max=100,
    step=1,
    description='Defect Size Max (voxels):',
    layout=Layout(width='100%')
)

defect_density_reduction = FloatSlider(
    value=0.3,
    min=0.0,
    max=0.8,
    step=0.05,
    description='Defect Density Reduction:',
    layout=Layout(width='100%')
)

noise_level = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Noise Level:',
    layout=Layout(width='100%')
)

ct_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_ct_button = Button(
    description='Preview CT Scan',
    button_style='info',
    icon='cube',
    layout=Layout(width='180px')
)

ct_config_panel = VBox([
    WidgetHTML("<b>CT Scan Configuration</b>"),
    WidgetHTML("<i>Grid Dimensions</i>"),
    ct_x_dim, ct_y_dim, ct_z_dim,
    WidgetHTML("<i>Voxel Spacing</i>"),
    ct_x_spacing, ct_y_spacing, ct_z_spacing,
    WidgetHTML("<i>Material Properties</i>"),
    base_density, density_variation,
    base_porosity, porosity_variation,
    WidgetHTML("<i>Defect Parameters</i>"),
    defect_probability, defect_size_min, defect_size_max,
    defect_density_reduction,
    WidgetHTML("<i>Scan Quality</i>"),
    noise_level,
    HBox([ct_random_seed, preview_ct_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- MongoDB Population Options ---
collections_to_populate = SelectMultiple(
    options=[
        ('STL Models', 'stl_models'),
        ('Hatching Layers', 'hatching_layers'),
        ('Laser Parameters', 'laser_parameters'),
        ('ISPM Data', 'ispm_data'),
        ('CT Scan Data', 'ct_scan_data')
    ],
    value=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    description='Collections:',
    layout=Layout(width='100%')
)

delete_all_toggle = Checkbox(
    value=False,
    description='Delete All Before Populate',
    layout=Layout(width='200px')
)

delete_by_model_id = Text(
    value='',
    placeholder='Enter model_id',
    description='Delete by Model ID:',
    layout=Layout(width='300px')
)

delete_by_model_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

delete_by_collection = Dropdown(
    options=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    value='stl_models',
    description='Delete Collection:',
    layout=Layout(width='200px')
)

delete_collection_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

batch_size = IntSlider(
    value=100,
    min=1,
    max=1000,
    step=10,
    description='Batch Size:',
    layout=Layout(width='100%')
)

parallel_workers = IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Parallel Workers:',
    layout=Layout(width='100%')
)

compress_data = Checkbox(
    value=False,
    description='Compress Data',
    layout=Layout(width='150px')
)

index_collections = Checkbox(
    value=True,
    description='Index Collections',
    layout=Layout(width='150px')
)

store_metadata = Checkbox(
    value=True,
    description='Store Metadata',
    layout=Layout(width='150px')
)

mongodb_config_panel = VBox([
    WidgetHTML("<b>MongoDB Population Options</b>"),
    collections_to_populate,
    WidgetHTML("<i>Delete Options</i>"),
    delete_all_toggle,
    HBox([delete_by_model_id, delete_by_model_button]),
    HBox([delete_by_collection, delete_collection_button]),
    WidgetHTML("<i>Batch Options</i>"),
    batch_size, parallel_workers,
    WidgetHTML("<i>Storage Options</i>"),
    HBox([compress_data, index_collections, store_metadata])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- Data Loading Section ---
load_model_dropdown = Dropdown(
    options=['Select a model to load...'],
    value='Select a model to load...',
    description='Load Model:',
    layout=Layout(width='300px')
)

load_model_button = Button(
    description='Load Data',
    button_style='primary',
    icon='download',
    layout=Layout(width='150px')
)

load_model_id_input = Text(
    value='',
    placeholder='Or enter model_id directly',
    description='Model ID:',
    layout=Layout(width='300px')
)

load_by_id_button = Button(
    description='Load by ID',
    button_style='info',
    icon='search',
    layout=Layout(width='150px')
)

load_status_display = WidgetHTML(value='<i>No data loaded</i>')

data_loading_panel = VBox([
    WidgetHTML("<b>Load Generated Data from MongoDB</b>"),
    HBox([load_model_dropdown, load_model_button]),
    HBox([load_model_id_input, load_by_id_button]),
    load_status_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


# Combine configuration panels into accordion
config_accordion = Accordion(children=[
    stl_config_panel,
    hatching_config_panel,
    laser_config_panel,
    ispm_config_panel,
    ct_config_panel,
    data_loading_panel,
    mongodb_config_panel
], selected_index=None)
config_accordion.set_title(0, 'STL Processing')
config_accordion.set_title(1, 'Hatching Generation')
config_accordion.set_title(2, 'Laser Parameters')
config_accordion.set_title(3, 'ISPM Data')
config_accordion.set_title(4, 'CT Scan')
config_accordion.set_title(5, 'Load Data')
config_accordion.set_title(6, 'MongoDB Population')
# ============================================
# Section 3: Center Panel - Visualization Tabs
# ============================================

# Create output widgets for each visualization tab
stl_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
hatching_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
laser_params_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ispm_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ct_scan_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
preview_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))

# Initialize with placeholder messages
with stl_output:
    display(HTML("<p><i>Load an STL file to see 3D voxel visualization</i></p>"))
with hatching_output:
    display(HTML("<p><i>Generate hatching data to see visualization</i></p>"))
with laser_params_output:
    display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))
with ispm_output:
    display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))
with ct_scan_output:
    display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))
with preview_output:
    display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))

# Create tab widget
visualization_tabs = Tab(children=[
    stl_output,
    hatching_output,
    laser_params_output,
    ispm_output,
    ct_scan_output,
    preview_output
])
visualization_tabs.set_title(0, 'STL')
visualization_tabs.set_title(1, 'Hatching')
visualization_tabs.set_title(2, 'Laser Parameters')
visualization_tabs.set_title(3, 'ISPM Data')
visualization_tabs.set_title(4, 'CT Scan')
visualization_tabs.set_title(5, 'Generation Preview')

# ============================================
# Section 4: Right Panel - Status & Results
# ============================================

# Generation status
current_operation = WidgetHTML(value='<b>Status:</b> Ready')
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Step progress indicators
step_status_html = WidgetHTML(value='''
<table style="width:100%">
<tr><td>STL Processing:</td><td>⏳</td></tr>
<tr><td>Hatching:</td><td>⏳</td></tr>
<tr><td>Laser Parameters:</td><td>⏳</td></tr>
<tr><td>ISPM Data:</td><td>⏳</td></tr>
<tr><td>CT Scan:</td><td>⏳</td></tr>
<tr><td>MongoDB Population:</td><td>⏳</td></tr>
</table>
''')

# Generation statistics
gen_stats_html = WidgetHTML(value='''
<b>Generation Statistics</b><br>
<i>No data generated yet</i>
''')

# MongoDB status
mongo_status_html = WidgetHTML(value='''
<b>MongoDB Status</b><br>
<i>Checking connection...</i>
''')

refresh_mongo_button = Button(
    description='Refresh Status',
    button_style='info',
    icon='refresh',
    layout=Layout(width='150px')
)

# Export options
export_config_button = Button(
    description='Export Config',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

export_preview_button = Button(
    description='Export Preview',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

export_stats_button = Button(
    description='Export Stats',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

# Right panel layout
status_panel = VBox([
    WidgetHTML("<b>Generation Status</b>"),
    current_operation,
    progress_bar,
    step_status_html,
    WidgetHTML("<hr>"),
    gen_stats_html,
    WidgetHTML("<hr>"),
    WidgetHTML("<b>MongoDB Status</b>"),
    mongo_status_html,
    refresh_mongo_button,
    WidgetHTML("<hr>"),
    WidgetHTML("<b>Export Options</b>"),
    export_config_button,
    export_preview_button,
    export_stats_button
], layout=Layout(
    padding='10px',
    border='1px solid #ccc',
    margin='5px',
    width='300px'
))

# ============================================
# Section 5: Bottom Panel - Status and Logs
# ============================================

bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

generation_logs = Output(layout=Layout(
    height='200px',
    border='1px solid #ccc',
    overflow_y='auto'
))

# Initialize logs
with generation_logs:
    display(HTML("<p><i>Generation logs will appear here...</i></p>"))

bottom_panel = VBox([
    bottom_status,
    bottom_progress,
    WidgetHTML("<b>Generation Logs:</b>"),
    generation_logs
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

# ============================================
# Complete Interface Layout Assembly
# ============================================

# Main layout: Left (config) + Center (visualization) + Right (status)
main_layout = HBox([
    # Left panel: Configuration accordion (25% width)
    VBox([config_accordion], layout=Layout(width='25%', padding='5px')),
    # Center panel: Visualization tabs (55% width)
    VBox([visualization_tabs], layout=Layout(width='55%', padding='5px')),
    # Right panel: Status and results (20% width)
    VBox([status_panel], layout=Layout(width='20%', padding='5px'))
], layout=Layout(
    width='100%',
    height='600px',
    border='2px solid #333',
    padding='10px'
))

# Complete interface
complete_interface = VBox([
    top_panel,
    main_layout,
    bottom_panel
], layout=Layout(width='100%'))

# ============================================================================
# EVENT HANDLER DEFINITIONS
# ============================================================================
# Define all event handlers that connect widgets to generation logic
# ============================================================================

# Generation start time
generation_start_time = None

# Helper Functions
def log_message(message: str, level: str = 'info'):
    """Log a message to the generation logs."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with generation_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display."""
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        elapsed = time.time() - (generation_start_time or time.time())
        bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'

def update_step_status(step: str, status: str):
    """Update step status indicator."""
    icons = {'pending': '⏳', 'running': '🔄', 'success': '✅', 'error': '❌'}
    icon = icons.get(status, '⏳')
    step_status_html.value = step_status_html.value.replace(
        f'<tr><td>{step}:</td><td>',
        f'<tr><td>{step}:</td><td>{icon}'
    )

def check_generation_ready() -> bool:
    """Check if generation is ready (STL selected)."""
    if not stl_selector.value or stl_selector.value == 'No STL files found' or stl_selector.value == 'Choose a model':
        return False
    return True

def update_generate_button_state():
    """Update Generate button enabled/disabled state based on readiness."""
    is_ready = check_generation_ready()
    generate_button.disabled = not is_ready
    if is_ready:
        generate_button.tooltip = 'Ready to generate all data types with default parameters'
        generate_button.button_style = 'success'
        generate_status_label.value = '<i style="color:green; font-size:0.9em;">✅ Ready to generate - will use default parameters</i>'
    else:
        generate_button.tooltip = 'Choose a model from the dropdown first'
        generate_button.button_style = ''
        generate_status_label.value = '<i style="color:gray; font-size:0.9em;">⚠️ Choose a model to enable Generate</i>'

def get_current_config() -> Dict[str, Any]:
    """Collect all current widget values into a configuration dictionary."""
    return {
        'generation_mode': generation_mode.value,
        'stl_file': stl_selector.value,
        'hatching': {
            'layer_thickness': layer_thickness.value,
            'hatch_spacing': hatch_spacing.value,
            'hatch_angle': hatch_angle.value,
            'contour_offset': contour_offset.value,
            'max_layers': max_layers.value,
            'sampling_spacing': sampling_spacing.value,
            'laser_beam_width': laser_beam_width.value,
            'pattern': hatch_pattern.value
        },
        'laser': {
            'power_mean': power_mean.value,
            'power_std': power_std.value,
            'power_min': power_min.value,
            'power_max': power_max.value,
            'speed_mean': speed_mean.value,
            'speed_std': speed_std.value,
            'speed_min': speed_min.value,
            'speed_max': speed_max.value,
            'hatch_spacing_mean': hatch_spacing_mean.value,
            'hatch_spacing_std': hatch_spacing_std.value,
            'contour_power_mult': contour_power_mult.value,
            'contour_speed_mult': contour_speed_mult.value,
            'support_power_mult': support_power_mult.value,
            'support_speed_mult': support_speed_mult.value,
            'random_seed': laser_random_seed.value if laser_random_seed.value else None
        },
        'ispm': {
            'base_temperature': base_temperature.value,
            'temperature_variation': temperature_variation.value,
            'melt_pool_width_mean': melt_pool_width_mean.value,
            'melt_pool_width_std': melt_pool_width_std.value,
            'melt_pool_length_mean': melt_pool_length_mean.value,
            'melt_pool_length_std': melt_pool_length_std.value,
            'melt_pool_depth_mean': melt_pool_depth_mean.value,
            'melt_pool_depth_std': melt_pool_depth_std.value,
            'cooling_rate_mean': cooling_rate_mean.value,
            'cooling_rate_std': cooling_rate_std.value,
            'temp_gradient_mean': temp_gradient_mean.value,
            'temp_gradient_std': temp_gradient_std.value,
            'sampling_rate': sampling_rate.value,
            'points_per_layer': points_per_layer.value,
            'random_seed': ispm_random_seed.value if ispm_random_seed.value else None
        },
        'ct_scan': {
            'grid_dimensions': (ct_x_dim.value, ct_y_dim.value, ct_z_dim.value),
            'voxel_spacing': (ct_x_spacing.value, ct_y_spacing.value, ct_z_spacing.value),
            'base_density': base_density.value,
            'density_variation': density_variation.value,
            'base_porosity': base_porosity.value,
            'porosity_variation': porosity_variation.value,
            'defect_probability': defect_probability.value,
            'defect_size_range': (defect_size_min.value, defect_size_max.value),
            'defect_density_reduction': defect_density_reduction.value,
            'noise_level': noise_level.value,
            'random_seed': ct_random_seed.value if ct_random_seed.value else None
        },
        'mongodb': {
            'collections': list(collections_to_populate.value),
            'delete_all': delete_all_toggle.value,
            'batch_size': batch_size.value,
            'parallel_workers': parallel_workers.value,
            'compress': compress_data.value,
            'index': index_collections.value,
            'store_metadata': store_metadata.value
        }
    }

# Event Handlers (full implementations - see updated on_generate_clicked in previous cell)
def on_stl_load_clicked(b):
    """Handle STL load button click."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    stl_filename = stl_selector.value
    if not stl_filename or stl_filename == 'No STL files found' or stl_filename == 'Choose a model':
        log_message("No STL file selected", 'warning')
        return
    try:
        log_message(f"Loading STL file: {stl_filename}", 'info')
        update_status("Loading STL...", 10)
        stl_processor = STLProcessor()
        stl_file_path = stl_processor.get_stl_file(stl_filename)
        if stl_file_path and stl_file_path.exists():
            try:
                import pyslm
                # Generate unique UUID for this model instance
                model_id = str(uuid.uuid4())
                model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
                # Create Part with ID, then set geometry (exact same pattern as populate_mongodb.py)
                stl_part = pyslm.Part(model_name)
                # Use str() directly, not resolve() to match populate_mongodb.py
                stl_file_str = str(stl_file_path)
                try:
                    stl_part.setGeometry(stl_file_str)
                    stl_part.origin = [0.0, 0.0, 0.0]
                    stl_part.rotation = [0, 0, 0]
                    stl_part.dropToPlatform()
                    # Access boundingBox after dropToPlatform (geometry should be set by now)
                    bbox = stl_part.boundingBox
                except Exception as geom_err:
                    error_msg = str(geom_err)
                    log_message(f"Error setting STL geometry: {error_msg}", 'error')
                    log_message(f"STL file path: {stl_file_str}", 'info')
                    raise  # Re-raise to be caught by outer exception handler
                metadata = {
                    'filename': stl_filename,
                    'path': str(stl_file_path),
                    'bounding_box': {
                        'x_min': bbox[0], 'x_max': bbox[3],
                        'y_min': bbox[1], 'y_max': bbox[4],
                        'z_min': bbox[2], 'z_max': bbox[5]
                    },
                    'volume': stl_part.volume if hasattr(stl_part, 'volume') else None,
                    'loaded_at': datetime.now().isoformat()
                }
                generated_data['stl_model'] = stl_part
                generated_data['model_id'] = model_id
                generated_data['model_name'] = model_name
                stl_metadata_display.value = json.dumps(metadata, indent=2)
                log_message(f"STL loaded successfully: {stl_filename}", 'success')
                update_status("STL loaded", 20)
                update_generate_button_state()
                if stl_preview_toggle.value:
                    with stl_output:
                        clear_output(wait=True)
                        display(HTML(f"<p><b>STL Model:</b> {stl_filename}</p>"))
                        display(HTML(f"<p><b>Bounding Box:</b> X: {bbox[0]:.2f} to {bbox[3]:.2f}, "
                                    f"Y: {bbox[1]:.2f} to {bbox[4]:.2f}, "
                                    f"Z: {bbox[2]:.2f} to {bbox[5]:.2f} mm</p>"))
                        if metadata['volume']:
                            display(HTML(f"<p><b>Volume:</b> {metadata['volume']:.2f} mm³</p>"))
                        display(HTML("<p><i>3D visualization would appear here (requires PyVista)</i></p>"))
            except ImportError:
                log_message("pyslm not available - limited STL processing", 'warning')
                metadata = {'filename': stl_filename, 'path': str(stl_file_path)}
                stl_metadata_display.value = json.dumps(metadata, indent=2)
                # Generate unique UUID for this model instance
                model_id = str(uuid.uuid4())
                model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
                generated_data['stl_model'] = {'path': str(stl_file_path)}
                generated_data['model_id'] = model_id
                generated_data['model_name'] = model_name
                update_generate_button_state()
        else:
            log_message(f"STL file not found: {stl_filename}", 'error')
            update_status("Error: STL file not found", 0)
    except Exception as e:
        log_message(f"Error loading STL: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

def on_preview_clicked(b):
    """Handle preview button click - visualize all loaded data."""
    log_message("🔄 Preview button clicked - visualizing all loaded data...", 'info')
    update_status("Generating visualizations...", 10)
    
    try:
        # Check what data is available
        has_hatching = generated_data.get('hatching_layers') is not None
        has_laser = generated_data.get('laser_parameters') is not None
        has_ispm = generated_data.get('ispm_data') is not None
        has_ct = generated_data.get('ct_scan') is not None
        
        if not any([has_hatching, has_laser, has_ispm, has_ct]):
            with preview_output:
                clear_output(wait=True)
                display(HTML("<p style='color:orange;'>⚠️ No data loaded. Please load data first using 'Load Data' button.</p>"))
            log_message("No data to visualize. Load data first.", 'warning')
            update_status("No data loaded", 0)
            return
        
        # Show summary in preview_output
        with preview_output:
            clear_output(wait=True)
            display(HTML("<h3>📊 Data Visualization Preview</h3>"))
            display(HTML("<p>Visualizing all loaded data in their respective tabs:</p>"))
            display(HTML("<ul>"))
            if has_hatching:
                display(HTML("<li>✅ Hatching data - Check 'STL & Hatching' tab</li>"))
            if has_laser:
                display(HTML("<li>✅ Laser parameters - Check 'Laser Parameters' tab</li>"))
            if has_ispm:
                display(HTML("<li>✅ ISPM data - Check 'ISPM Data' tab</li>"))
            if has_ct:
                display(HTML("<li>✅ CT scan - Check 'CT Scan' tab</li>"))
            display(HTML("</ul>"))

        update_status("Visualizing STL...", 25)
        # Visualize STL voxel image first (if STL is loaded)
        has_stl = generated_data.get('stl_model') is not None
        if has_stl:
            visualize_stl_voxel_image()

        update_status("Visualizing hatching...", 25)
        # Visualize hatching if available
        if has_hatching:
            visualize_hatching_with_pyslm()
        
        update_status("Visualizing laser parameters...", 50)
        # Visualize laser parameters if available
        if has_laser:
            visualize_laser_parameters()
        
        update_status("Visualizing ISPM data...", 75)
        # Visualize ISPM data if available
        if has_ispm:
            visualize_ispm_data()
        
        update_status("Visualizing CT scan...", 90)
        # Visualize CT scan if available
        if has_ct:
            visualize_ct_scan()
        
        update_status("Visualizations complete!", 100)
        log_message("✅ All visualizations generated successfully", 'success')
        
    except Exception as e:
        log_message(f"Error generating visualizations: {str(e)}", 'error')
        import traceback
        with preview_output:
            clear_output(wait=True)
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
        update_status(f"Error: {str(e)}", 0)

def on_generate_clicked(b):
    """Handle generate button click - full data generation and MongoDB population."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    if not check_generation_ready():
        log_message("Please select an STL file first", 'warning')
        return
    global generation_start_time
    generation_start_time = time.time()
    stl_filename = stl_selector.value
    
    # Skip if placeholder is selected
    if stl_filename == 'Choose a model' or stl_filename == 'No STL files found':
        log_message("Please select an STL file from the dropdown", 'warning')
        return
    
    # Auto-load STL if not already loaded
    if generated_data.get('stl_model') is None:
        log_message(f"Auto-loading STL file: {stl_filename}", 'info')
        update_status("Loading STL...", 5)
        try:
            stl_processor = STLProcessor()
            stl_file_path = stl_processor.get_stl_file(stl_filename)
            if stl_file_path and stl_file_path.exists():
                try:
                    import pyslm
                    # Generate unique UUID for this model instance
                    model_id = str(uuid.uuid4())
                    model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
                    # Create Part with ID, then set geometry (exact same pattern as populate_mongodb.py)
                    stl_part = pyslm.Part(model_name)
                    # Use str() directly, not resolve() to match populate_mongodb.py exactly
                    stl_file_str = str(stl_file_path)
                    try:
                        stl_part.setGeometry(stl_file_str)
                        stl_part.origin = [0.0, 0.0, 0.0]
                        stl_part.rotation = [0, 0, 0]
                        stl_part.dropToPlatform()
                        generated_data['stl_model'] = stl_part
                        generated_data['model_id'] = model_id
                        generated_data['model_name'] = model_name
                        log_message("STL loaded successfully", 'success')
                    except Exception as geom_err:
                        error_msg = str(geom_err)
                        log_message(f"Error setting STL geometry: {error_msg}", 'error')
                        log_message(f"STL file path: {stl_file_str}", 'info')
                        log_message(f"File exists: {stl_file_path.exists()}", 'info')
                        raise  # Re-raise to be caught by outer exception handler
                except ImportError:
                    # Generate unique UUID for this model instance
                    model_id = str(uuid.uuid4())
                    model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
                    generated_data['stl_model'] = {'path': str(stl_file_path)}
                    generated_data['model_id'] = model_id
                    generated_data['model_name'] = model_name
                    log_message("STL file found (pyslm not available)", 'warning')
            else:
                log_message(f"STL file not found: {stl_filename}", 'error')
                update_status("Error: STL file not found", 0)
                return
        except Exception as e:
            log_message(f"Error loading STL: {str(e)}", 'error')
            import traceback
            log_message(f"Traceback: {traceback.format_exc()}", 'error')
            update_status(f"Error: {str(e)}", 0)
            return
    
    # Get collections to populate
    collections = list(collections_to_populate.value) if len(collections_to_populate.value) > 0 else [
        'stl_models', 'hatching_layers', 'laser_parameters', 'ispm_monitoring_data', 'ct_scan_data'
    ]
    
    log_message(f"Starting full data generation for: {stl_filename}", 'info')
    log_message(f"Will generate: {', '.join(collections)}", 'info')
    update_status("Generating data...", 10)
    
    try:
        # Ensure model_id is set (should be set when STL is loaded)
        if not generated_data.get('model_id'):
            # Generate unique UUID if not already set
            model_id = str(uuid.uuid4())
            model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
            generated_data['model_id'] = model_id
            generated_data['model_name'] = model_name
        else:
            model_id = generated_data['model_id']
            model_name = generated_data.get('model_name', stl_filename.replace('.stl', ''))
        config = get_current_config()
        
        # Get STL bounding box
        stl_part = generated_data.get('stl_model')
        if not stl_part:
            log_message("STL model not loaded", 'error')
            return
        
        try:
            import pyslm
            if isinstance(stl_part, pyslm.Part):
                bbox = stl_part.boundingBox
                bbox_dict = {'min': (bbox[0], bbox[1], bbox[2]), 'max': (bbox[3], bbox[4], bbox[5])}
                n_layers = int((bbox[5] - bbox[2]) / config['hatching']['layer_thickness'])
            else:
                bbox_dict = {'min': (-50, -50, 0), 'max': (50, 50, 10)}
                n_layers = 20
        except:
            bbox_dict = {'min': (-50, -50, 0), 'max': (50, 50, 10)}
            n_layers = 20
        
        update_step_status("STL Processing", "success")
        
        # Generate hatching
        hatching_result = None
        if 'hatching_layers' in collections:
            update_step_status("Hatching", "running")
            update_status("Generating hatching paths...", 30)
            if GENERATION_AVAILABLE:
                try:
                    import pyslm
                    if isinstance(stl_part, pyslm.Part):
                        # Ensure geometry is set (same pattern as populate_mongodb.py)
                        if not hasattr(stl_part, 'geometry') or stl_part.geometry is None:
                            stl_processor = STLProcessor()
                            stl_file_path = stl_processor.get_stl_file(stl_filename)
                            if stl_file_path and stl_file_path.exists():
                                # Use str() directly to match populate_mongodb.py
                                stl_file_str = str(stl_file_path)
                                stl_part.setGeometry(stl_file_str)
                                stl_part.origin = [0.0, 0.0, 0.0]
                                stl_part.rotation = [0, 0, 0]
                                stl_part.dropToPlatform()
                        
                        # Verify geometry is set before generating
                        if hasattr(stl_part, 'geometry') and stl_part.geometry is not None:
                            # Explicitly create config with max_layers=None to ensure all layers are generated
                            hatching_config = HatchingConfig(
                                layer_thickness=config['hatching']['layer_thickness'],
                                hatch_spacing=config['hatching']['hatch_spacing'],
                                hatch_angle=config['hatching']['hatch_angle'],
                                contour_offset=config['hatching']['contour_offset'],
                                max_layers=None,  # Generate all layers
                                sampling_spacing=config['hatching']['sampling_spacing'],
                                laser_beam_width=config['hatching']['laser_beam_width']
                            )
                            hatching_gen = HatchingGenerator(config=hatching_config)

                            #hatching_gen = HatchingGenerator()
                            hatching_result = hatching_gen.generate_hatching(stl_part)
                            generated_data['hatching_data'] = hatching_result
                            log_message(f"Hatching generated: {len(hatching_result.get('layers', []))} layers", 'success')
                            update_step_status("Hatching", "success")
                        else:
                            log_message("STL geometry not set - cannot generate hatching", 'error')
                            update_step_status("Hatching", "error")
                    else:
                        log_message("Hatching requires pyslm.Part object", 'warning')
                        update_step_status("Hatching", "error")
                except ImportError:
                    log_message("pyslm not available - cannot generate hatching", 'warning')
                    update_step_status("Hatching", "error")
                except Exception as e:
                    log_message(f"Error generating hatching: {str(e)}", 'error')
                    import traceback
                    log_message(f"Traceback: {traceback.format_exc()}", 'error')
                    update_step_status("Hatching", "error")
            else:
                log_message("Generation modules not available", 'warning')
                update_step_status("Hatching", "error")
        
        # Generate laser parameters
        laser_data = None
        if 'laser_parameters' in collections:
            update_step_status("Laser Parameters", "running")
            update_status("Generating laser parameters...", 50)
            if GENERATION_AVAILABLE:
                try:
                    from generation.sensors.laser_parameter_generator import LaserParameterGeneratorConfig
                    laser_config = LaserParameterGeneratorConfig(
                        power_mean=config['laser']['power_mean'], power_std=config['laser']['power_std'],
                        power_min=config['laser']['power_min'], power_max=config['laser']['power_max'],
                        speed_mean=config['laser']['speed_mean'], speed_std=config['laser']['speed_std'],
                        speed_min=config['laser']['speed_min'], speed_max=config['laser']['speed_max'],
                        hatch_spacing_mean=config['laser']['hatch_spacing_mean'],
                        hatch_spacing_std=config['laser']['hatch_spacing_std'],
                        contour_power_multiplier=config['laser']['contour_power_mult'],
                        contour_speed_multiplier=config['laser']['contour_speed_mult'],
                        support_power_multiplier=config['laser']['support_power_mult'],
                        support_speed_multiplier=config['laser']['support_speed_mult'],
                        random_seed=config['laser']['random_seed']
                    )
                    laser_gen = LaserParameterGenerator(config=laser_config)
                    # generate_for_build signature: (build_id, n_layers, layer_thickness, points_per_layer, bounding_box)
                    layer_thickness = config['hatching']['layer_thickness']
                    points_per_layer = 1000
                    laser_data = laser_gen.generate_for_build(
                        build_id=model_id,
                        n_layers=n_layers,
                        layer_thickness=layer_thickness,
                        points_per_layer=points_per_layer,
                        bounding_box=bbox_dict
                    )
                    generated_data['laser_params'] = laser_data
                    log_message(f"Laser parameters generated: {len(laser_data.get('points', []))} points", 'success')
                    update_step_status("Laser Parameters", "success")
                except Exception as e:
                    log_message(f"Error generating laser parameters: {str(e)}", 'error')
                    import traceback
                    log_message(f"Traceback: {traceback.format_exc()}", 'error')
                    update_step_status("Laser Parameters", "error")
        
        # Generate ISPM data
        ispm_data = None
        if 'ispm_monitoring_data' in collections or 'ispm_data' in collections:
            update_step_status("ISPM Data", "running")
            update_status("Generating ISPM data...", 70)
            if GENERATION_AVAILABLE:
                try:
                    from generation.sensors.ispm_generator import ISPMGeneratorConfig
                    ispm_config = ISPMGeneratorConfig(
                        base_temperature=config['ispm']['base_temperature'],
                        temperature_variation=config['ispm']['temperature_variation'],
                        melt_pool_width_mean=config['ispm']['melt_pool_width_mean'],
                        melt_pool_width_std=config['ispm']['melt_pool_width_std'],
                        melt_pool_length_mean=config['ispm']['melt_pool_length_mean'],
                        melt_pool_length_std=config['ispm']['melt_pool_length_std'],
                        melt_pool_depth_mean=config['ispm']['melt_pool_depth_mean'],
                        melt_pool_depth_std=config['ispm']['melt_pool_depth_std'],
                        cooling_rate_mean=config['ispm']['cooling_rate_mean'],
                        cooling_rate_std=config['ispm']['cooling_rate_std'],
                        temperature_gradient_mean=config['ispm']['temp_gradient_mean'],
                        temperature_gradient_std=config['ispm']['temp_gradient_std'],
                        sampling_rate=config['ispm']['sampling_rate'],
                        points_per_layer=config['ispm']['points_per_layer'],
                        random_seed=config['ispm']['random_seed']
                    )
                    ispm_gen = ISPMGenerator(config=ispm_config)
                    # ISPM generator expects {'x': (min, max), 'y': (min, max), 'z': (min, max)} format
                    ispm_bbox = {
                        'x': (bbox_dict['min'][0], bbox_dict['max'][0]),
                        'y': (bbox_dict['min'][1], bbox_dict['max'][1]),
                        'z': (bbox_dict['min'][2], bbox_dict['max'][2])
                    }
                    layer_thickness = config['hatching']['layer_thickness']
                    ispm_data = ispm_gen.generate_for_build(
                        build_id=model_id,
                        n_layers=n_layers,
                        layer_thickness=layer_thickness,
                        bounding_box=ispm_bbox
                    )
                    generated_data['ispm_data'] = ispm_data
                    log_message(f"ISPM data generated: {len(ispm_data.get('data_points', []))} points", 'success')
                    update_step_status("ISPM Data", "success")
                except Exception as e:
                    log_message(f"Error generating ISPM data: {str(e)}", 'error')
                    import traceback
                    log_message(f"Traceback: {traceback.format_exc()}", 'error')
                    update_step_status("ISPM Data", "error")
        
        # Generate CT scan
        ct_data = None
        if 'ct_scan_data' in collections:
            update_step_status("CT Scan", "running")
            update_status("Generating CT scan data...", 85)
            if GENERATION_AVAILABLE:
                try:
                    from generation.sensors.ct_scan_generator import CTScanGeneratorConfig
                    ct_config = CTScanGeneratorConfig(
                        grid_dimensions=config['ct_scan']['grid_dimensions'],
                        voxel_spacing=config['ct_scan']['voxel_spacing'],
                        base_density=config['ct_scan']['base_density'],
                        density_variation=config['ct_scan']['density_variation'],
                        base_porosity=config['ct_scan']['base_porosity'],
                        porosity_variation=config['ct_scan']['porosity_variation'],
                        defect_probability=config['ct_scan']['defect_probability'],
                        defect_size_range=config['ct_scan']['defect_size_range'],
                        defect_density_reduction=config['ct_scan']['defect_density_reduction'],
                        noise_level=config['ct_scan']['noise_level'],
                        random_seed=config['ct_scan']['random_seed']
                    )
                    ct_gen = CTScanGenerator(config=ct_config)
                    ct_bbox = {'x': (bbox_dict['min'][0], bbox_dict['max'][0]),
                              'y': (bbox_dict['min'][1], bbox_dict['max'][1]),
                              'z': (bbox_dict['min'][2], bbox_dict['max'][2])}
                    ct_data = ct_gen.generate_for_build(model_id, bounding_box=ct_bbox)
                    generated_data['ct_scan'] = ct_data
                    log_message(f"CT scan generated: {len(ct_data.get('defect_locations', []))} defects", 'success')
                    update_step_status("CT Scan", "success")
                except Exception as e:
                    log_message(f"Error generating CT scan: {str(e)}", 'error')
                    import traceback
                    log_message(f"Traceback: {traceback.format_exc()}", 'error')
                    update_step_status("CT Scan", "error")
        
        # Populate MongoDB
        if mongo_client and mongo_client.is_connected():
            update_step_status("MongoDB Population", "running")
            update_status("Populating MongoDB...", 90)
            try:
                import uuid, io, gzip
                from gridfs import GridFS
                db = mongo_client.database
                fs = GridFS(db)
                # Ensure model_id is set (should be set when STL is loaded)
                if not generated_data.get('model_id'):
                    model_id = str(uuid.uuid4())
                    model_name = f"{stl_filename.replace('.stl', '')}_{uuid.uuid4().hex[:8]}"
                    generated_data['model_id'] = model_id
                    generated_data['model_name'] = model_name
                else:
                    model_id = generated_data['model_id']
                    model_name = generated_data.get('model_name', stl_filename.replace('.stl', ''))
                
                if 'stl_models' in collections:
                    db['stl_models'].insert_one({
                        'model_id': model_id, 'model_name': model_name,
                        'filename': stl_filename, 'metadata': {'bounding_box': bbox_dict, 'n_layers': n_layers},
                        'created_at': datetime.now().isoformat()
                    })
                    log_message("STL model metadata stored", 'success')

                hatching_result = generated_data.get('hatching_data')
                if 'hatching_layers' in collections and hatching_result:
                    # Extract points, power, velocity, energy from result (same as populate_mongodb.py)
                    all_points = hatching_result.get('points', np.array([]))
                    all_power = hatching_result.get('power', np.array([]))
                    all_velocity = hatching_result.get('velocity', np.array([]))
                    all_energy = hatching_result.get('energy', np.array([]))
    
                    # Get laser beam parameters from metadata
                    metadata = hatching_result.get('metadata', {})
                    laser_beam_width = metadata.get('laser_beam_width', 0.1)
                    hatch_spacing = metadata.get('hatch_spacing', 0.1)
                    overlap_percentage = metadata.get('overlap_percentage', 0.0)
                    overlap_ratio = metadata.get('overlap_ratio', 0.0)
                    coordinate_system = metadata.get('coordinate_system', None)
    
                    # Group points by layer and create hatch paths
                    hatching_docs = []
                    if len(all_points) > 0:
                        for layer_idx, layer in enumerate(hatching_result.get('layers', [])):
                            z_height = layer.z / 1000.0  # Convert from microns to mm
                            z_tolerance = 0.001  # 1 micron tolerance
            
                            # Find all points for this layer (matching z-coordinate)
                            layer_mask = np.abs(all_points[:, 2] - z_height) < z_tolerance
                            layer_points = all_points[layer_mask]
                            layer_power = all_power[layer_mask] if len(all_power) > 0 else np.array([])
                            layer_velocity = all_velocity[layer_mask] if len(all_velocity) > 0 else np.array([])
                            layer_energy = all_energy[layer_mask] if len(all_energy) > 0 else np.array([])
            
                            # Group points into hatch paths (sequential points form a hatch)
                            hatches = []
                            if len(layer_points) > 0:
                                # Group consecutive points into hatch segments
                                current_hatch = []
                                current_power = layer_power[0] if len(layer_power) > 0 else 200.0
                                current_velocity = layer_velocity[0] if len(layer_velocity) > 0 else 500.0
                
                                for i, point in enumerate(layer_points):
                                    if len(current_hatch) == 0:
                                        current_hatch = [point.tolist()]
                                        if i < len(layer_power):
                                            current_power = float(layer_power[i])
                                        if i < len(layer_velocity):
                                            current_velocity = float(layer_velocity[i])
                                    else:
                                        # Check if point continues the current hatch (within tolerance)
                                        last_point = current_hatch[-1]
                                        distance = np.linalg.norm(np.array(point[:2]) - np.array(last_point[:2]))
                        
                                        if distance < 2.0:  # Points within 2mm are part of same hatch
                                            current_hatch.append(point.tolist())
                                        else:
                                            # End current hatch, start new one
                                            if len(current_hatch) > 1:
                                                hatches.append({
                                                    'start_point': current_hatch[0],
                                                    'end_point': current_hatch[-1],
                                                    'points': current_hatch,  # Full path coordinates X, Y, Z
                                                    'laser_power': current_power,
                                                    'scan_speed': current_velocity,
                                                    'energy_density': float(layer_energy[i-1]) if i-1 < len(layer_energy) else current_power / (current_velocity * 0.1),
                                                    'laser_beam_width': laser_beam_width,
                                                    'hatch_spacing': hatch_spacing,
                                                    'overlap_percentage': overlap_percentage,
                                                    'hatch_type': 'raster',
                                                    'scan_order': len(hatches)
                                                })
                                            current_hatch = [point.tolist()]
                                            if i < len(layer_power):
                                                current_power = float(layer_power[i])
                                            if i < len(layer_velocity):
                                                current_velocity = float(layer_velocity[i])
                
                                # Add final hatch
                                if len(current_hatch) > 1:
                                    hatches.append({
                                        'start_point': current_hatch[0],
                                        'end_point': current_hatch[-1],
                                        'points': current_hatch,  # Full path coordinates X, Y, Z
                                        'laser_power': current_power,
                                        'scan_speed': current_velocity,
                                        'energy_density': float(layer_energy[-1]) if len(layer_energy) > 0 else current_power / (current_velocity * 0.1),
                                        'laser_beam_width': laser_beam_width,
                                        'hatch_spacing': hatch_spacing,
                                        'overlap_percentage': overlap_percentage,
                                        'hatch_type': 'raster',
                                        'scan_order': len(hatches)
                                    })
            
                            hatching_docs.append({
                                'model_id': model_id,
                                'layer_index': layer_idx,
                                'layer_height': metadata.get('layer_thickness', 0.05),
                                'z_position': z_height,
                                'contours': [],  # Contours can be extracted separately if needed
                                'hatches': hatches,  # Laser path coordinates with parameters - THIS IS THE KEY FIELD!
                                'processing_time': datetime.now().isoformat(),
                                'coordinate_system': coordinate_system,
                                'metadata': {
                                    'n_contours': 0,
                                    'n_hatches': len(hatches),
                                    'n_points': len(layer_points),
                                    'hatch_spacing': hatch_spacing,
                                    'laser_beam_width': laser_beam_width,
                                    'overlap_percentage': overlap_percentage,
                                    'overlap_ratio': overlap_ratio
                                }
                            })
    
                    if hatching_docs:
                        db['hatching_layers'].insert_many(hatching_docs)
                        log_message(f"Hatching data stored: {len(hatching_docs)} layers with {sum(len(doc.get('hatches', [])) for doc in hatching_docs)} hatches", 'success')
        
                if 'laser_parameters' in collections and laser_data:
                    laser_docs = [{'model_id': model_id, 'layer_index': p.layer_index,
                                  'spatial_coordinates': [p.x, p.y, p.z], 'laser_power': p.laser_power,
                                  'scan_speed': p.scan_speed, 'energy_density': p.energy_density,
                                  'timestamp': p.timestamp.isoformat()}
                                 for p in laser_data.get('points', [])]
                    batch_size = config['mongodb']['batch_size']
                    for i in range(0, len(laser_docs), batch_size):
                        db['laser_parameters'].insert_many(laser_docs[i:i+batch_size])
                    log_message(f"Laser parameters stored: {len(laser_docs)} points", 'success')
                
                collection_name = 'ispm_monitoring_data'
                if (collection_name in collections or 'ispm_data' in collections) and ispm_data:
                    ispm_docs = [{'model_id': model_id, 'layer_index': p.layer_index,
                                 'spatial_coordinates': [p.x, p.y, p.z], 'melt_pool_temperature': p.melt_pool_temperature,
                                 'melt_pool_size': p.melt_pool_size, 'peak_temperature': p.peak_temperature,
                                 'cooling_rate': p.cooling_rate, 'temperature_gradient': p.temperature_gradient,
                                 'timestamp': p.timestamp.isoformat()}
                                for p in ispm_data.get('data_points', [])]
                    batch_size = config['mongodb']['batch_size']
                    for i in range(0, len(ispm_docs), batch_size):
                        db[collection_name].insert_many(ispm_docs[i:i+batch_size])
                    log_message(f"ISPM data stored: {len(ispm_docs)} points", 'success')
                
                if 'ct_scan_data' in collections and ct_data:
                    voxel_grid = ct_data.get('voxel_grid')
                    defect_locations = ct_data.get('defect_locations', [])
                    density_gridfs_id = porosity_gridfs_id = None
                    if voxel_grid and hasattr(voxel_grid, 'density_values'):
                        try:
                            buffer = io.BytesIO()
                            np.save(buffer, voxel_grid.density_values)
                            density_gridfs_id = fs.put(gzip.compress(buffer.getvalue()), filename=f"{model_id}_density.npy.gz")
                        except: pass
                    if voxel_grid and hasattr(voxel_grid, 'porosity_map'):
                        try:
                            buffer = io.BytesIO()
                            np.save(buffer, voxel_grid.porosity_map)
                            porosity_gridfs_id = fs.put(gzip.compress(buffer.getvalue()), filename=f"{model_id}_porosity.npy.gz")
                        except: pass
                    
                    # Process metadata - convert dataclass objects to dicts for MongoDB serialization
                    # (same approach as populate_mongodb.py)
                    from dataclasses import asdict, is_dataclass
                    raw_metadata = ct_data.get('metadata', {})
                    processed_metadata = {}
                    for key, value in raw_metadata.items():
                        if hasattr(value, '__dict__') and not isinstance(value, (dict, list, str, int, float, bool, type(None))):
                            # Convert dataclass to dict
                            if is_dataclass(value):
                                processed_metadata[key] = asdict(value)
                            else:
                                # Skip non-serializable objects
                                processed_metadata[key] = str(value)
                        else:
                            processed_metadata[key] = value
                    
                    db['ct_scan_data'].insert_one({
                        'model_id': model_id,
                        'voxel_grid': {'dimensions': list(voxel_grid.dimensions) if voxel_grid else None,
                                      'spacing': list(voxel_grid.spacing) if voxel_grid else None,
                                      'origin': list(voxel_grid.origin) if voxel_grid else None},
                        'data_storage': {'density_values_gridfs_id': str(density_gridfs_id) if density_gridfs_id else None,
                                        'porosity_map_gridfs_id': str(porosity_gridfs_id) if porosity_gridfs_id else None},
                        'defect_locations': defect_locations[:1000], 'defect_count': len(defect_locations),
                        'metadata': processed_metadata
                    })
                    log_message(f"CT scan data stored: {len(defect_locations)} defects", 'success')
                
                update_step_status("MongoDB Population", "success")
                log_message("MongoDB population complete!", 'success')
            except Exception as e:
                log_message(f"Error populating MongoDB: {str(e)}", 'error')
                update_step_status("MongoDB Population", "error")
        else:
            log_message("MongoDB not connected - skipping population", 'warning')
        
        log_message("Data generation complete!", 'success')
        update_status("Generation complete!", 100)
        on_refresh_mongo_clicked(None)
    except Exception as e:
        log_message(f"Error during generation: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

def on_clear_preview_clicked(b):
    """Clear all preview visualizations."""
    with stl_output:
        clear_output(wait=True)
        display(HTML("<p><i>Load an STL file to see 3D voxel visualization</i></p>"))
    with hatching_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate hatching data to see visualization</i></p>"))
    with laser_params_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))
    with ispm_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))
    with ct_scan_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))
    with preview_output:
        clear_output(wait=True)
        display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))
    log_message("Preview cleared", 'info')

def on_save_config_clicked(b):
    """Save current configuration to JSON file."""
    try:
        config = get_current_config()
        filename = f"generation_config_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        config_path = project_root / 'generation' / 'configs' / filename
        config_path.parent.mkdir(parents=True, exist_ok=True)
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
        log_message(f"Configuration saved: {filename}", 'success')
        with preview_output:
            clear_output(wait=True)
            display(HTML(f'<p>✅ Configuration saved: <code>{filename}</code></p>'))
    except Exception as e:
        log_message(f"Error saving configuration: {str(e)}", 'error')

def on_refresh_mongo_clicked(b):
    """Refresh MongoDB status display."""
    if not mongo_client:
        mongo_status_html.value = '<b>MongoDB Status</b><br><span style="color:red;">Not connected</span>'
        return
    if not mongo_client.is_connected():
        mongo_status_html.value = '<b>MongoDB Status</b><br><span style="color:red;">Not connected</span>'
        return
    try:
        db = mongo_client.database
        db_name = mongo_client.config.database
        collections = db.list_collection_names()
        status_text = '<b>MongoDB Status</b><br>'
        status_text += f'<b>Database:</b> {db_name}<br>'
        status_text += f'<b>Collections:</b> {len(collections)}<br><ul>'
        for coll_name in collections[:10]:
            count = db[coll_name].count_documents({})
            status_text += f'<li>{coll_name}: {count} documents</li>'
        if len(collections) > 10:
            status_text += f'<li>... and {len(collections) - 10} more</li>'
        status_text += '</ul>'
        mongo_status_html.value = status_text
        log_message("MongoDB status refreshed", 'success')
    except Exception as e:
        mongo_status_html.value = f'<b>MongoDB Status</b><br><span style="color:red;">Error: {str(e)}</span>'
        log_message(f"Error refreshing MongoDB status: {str(e)}", 'error')

def on_collections_change(change):
    """Update Generate button when collections selection changes."""
    update_generate_button_state()

def on_stl_selector_change(change):
    """Update Generate button when STL selection changes."""
    update_generate_button_state# ============================================================================
# DATA LOADING AND VISUALIZATION FUNCTIONS
# ============================================================================

def refresh_model_dropdown():
    """Refresh the model dropdown with available models from MongoDB."""
    if not mongo_client or not mongo_client.is_connected():
        load_model_dropdown.options = ['Select a model to load...']
        return
    
    try:
        db = mongo_client.database
        models = list(db['stl_models'].find({}, {'model_id': 1, 'model_name': 1, 'filename': 1}).limit(100))
        
        options = ['Select a model to load...']
        for model in models:
            model_id = model.get('model_id', 'Unknown')
            model_name = model.get('model_name') or model.get('filename', 'Unknown')
            display_name = f"{model_name} ({model_id[:8]}...)"
            options.append((display_name, model_id))
        
        load_model_dropdown.options = options
        log_message(f"Found {len(models)} models in MongoDB", 'success')
    except Exception as e:
        log_message(f"Error refreshing model list: {str(e)}", 'error')
        load_model_dropdown.options = ['Select a model to load...']

def load_data_from_mongodb(model_id):
    """Load all data for a given model_id from MongoDB."""
    if not mongo_client or not mongo_client.is_connected():
        log_message("MongoDB not connected", 'error')
        load_status_display.value = '<span style="color:red;">❌ MongoDB not connected</span>'
        return None
    
    try:
        db = mongo_client.database
        loaded_data = {
            'model_id': model_id,
            'stl_model': None,
            'hatching_layers': None,
            'laser_parameters': None,
            'ispm_data': None,
            'ct_scan': None
        }
        
        # Load STL model metadata
        stl_doc = db['stl_models'].find_one({'model_id': model_id})
        if stl_doc:
            loaded_data['stl_model'] = stl_doc
            log_message(f"Loaded STL model: {stl_doc.get('model_name', 'Unknown')}", 'success')
        
        # Load hatching layers
        hatching_docs = list(db['hatching_layers'].find({'model_id': model_id}).sort('layer_index', 1))
        if hatching_docs:
            loaded_data['hatching_layers'] = hatching_docs
            log_message(f"Loaded {len(hatching_docs)} hatching layers", 'success')
        
        # Load laser parameters
        laser_docs = list(db['laser_parameters'].find({'model_id': model_id}).limit(10000))
        if laser_docs:
            loaded_data['laser_parameters'] = laser_docs
            log_message(f"Loaded {len(laser_docs)} laser parameter points", 'success')
        
        # Load ISPM data
        ispm_docs = list(db['ispm_monitoring_data'].find({'model_id': model_id}).limit(10000))
        if ispm_docs:
            loaded_data['ispm_data'] = ispm_docs
            log_message(f"Loaded {len(ispm_docs)} ISPM data points", 'success')
        
        # Load CT scan data
        ct_doc = db['ct_scan_data'].find_one({'model_id': model_id})
        if ct_doc:
            # Load GridFS data if available
            if ct_doc.get('data_storage', {}).get('density_values_gridfs_id'):
                try:
                    from gridfs import GridFS
                    import gzip, io
                    fs = GridFS(db)
                    gridfs_id = ct_doc['data_storage']['density_values_gridfs_id']
                    density_data = fs.get(gridfs_id).read()
                    density_data = gzip.decompress(density_data)
                    density_values = np.load(io.BytesIO(density_data))
                    ct_doc['density_values'] = density_values
                except Exception as e:
                    log_message(f"Could not load CT density data: {str(e)}", 'warning')
            
            loaded_data['ct_scan'] = ct_doc
            log_message("Loaded CT scan data", 'success')
        
        # Update generated_data with loaded data
        generated_data.update(loaded_data)
        
        # Update status
        status_text = f'<b>✅ Loaded data for model: {model_id[:8]}...</b><br>'
        status_text += f'STL: {"✅" if loaded_data["stl_model"] else "❌"} | '
        status_text += f'Hatching: {len(hatching_docs) if hatching_docs else 0} layers | '
        status_text += f'Laser: {len(laser_docs) if laser_docs else 0} points | '
        status_text += f'ISPM: {len(ispm_docs) if ispm_docs else 0} points | '
        status_text += f'CT: {"✅" if loaded_data["ct_scan"] else "❌"}'
        load_status_display.value = status_text
        
        return loaded_data
        
    except Exception as e:
        log_message(f"Error loading data: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        load_status_display.value = f'<span style="color:red;">❌ Error: {str(e)}</span>'
        return None

def on_load_model_clicked(b):
    """Handle load model button click."""
    model_id = load_model_dropdown.value
    if model_id == 'Select a model to load...':
        log_message("Please select a model from the dropdown", 'warning')
        return
    
    log_message(f"Loading data for model: {model_id[:8]}...", 'info')
    load_data_from_mongodb(model_id)

def on_load_by_id_clicked(b):
    """Handle load by ID button click."""
    model_id = load_model_id_input.value.strip()
    if not model_id:
        log_message("Please enter a model_id", 'warning')
        return
    
    log_message(f"Loading data for model_id: {model_id[:8]}...", 'info')
    load_data_from_mongodb(model_id)

def visualize_stl_voxel_image():
    """Display STL 3D surface geometry - the model used for data generation."""
    with stl_output:
        clear_output(wait=True)
        display(HTML("<p>🔄 Loading STL geometry...</p>"))
        
        stl_model = generated_data.get('stl_model')
        
        if not stl_model:
            display(HTML("<p style='color:orange;'>⚠️ No STL model loaded. Load STL file first.</p>"))
            return
        
        try:
            import pyvista as pv
            import numpy as np
            PYVISTA_AVAILABLE = True
        except ImportError:
            display(HTML("<p style='color:red;'>❌ PyVista not available. Install: pip install pyvista</p>"))
            return
        
        try:
            from generation.process.stl_processor import STLProcessor
            from pathlib import Path
            
            # Load STL mesh (no voxelization - direct surface rendering)
            mesh = None
            
            # Handle different STL model formats
            if isinstance(stl_model, dict):
                filename = stl_model.get('filename', '')
                if filename:
                    try:
                        stl_processor = STLProcessor()
                        stl_file_path = stl_processor.get_stl_file(filename)
                        if stl_file_path and stl_file_path.exists():
                            mesh = pv.read(str(stl_file_path))
                    except Exception as e:
                        log_message(f"Error getting STL file path: {str(e)}", 'warning')
                
                # Try direct path
                if mesh is None and 'path' in stl_model:
                    stl_file_path = Path(stl_model['path'])
                    if stl_file_path.exists():
                        mesh = pv.read(str(stl_file_path))
            
            elif hasattr(stl_model, 'geometry'):
                # pyslm.Part object - convert geometry to PyVista mesh
                try:
                    import trimesh
                    trimesh_mesh = stl_model.geometry
                    vertices = np.array(trimesh_mesh.vertices)
                    faces = np.array(trimesh_mesh.faces)
                    # Convert to PyVista format: [n, v1, v2, v3, ...] where n=3 for triangles
                    faces_pv = np.column_stack([np.full(len(faces), 3), faces]).flatten()
                    mesh = pv.PolyData(vertices, faces_pv)
                except Exception as e:
                    log_message(f"Error converting pyslm geometry: {str(e)}", 'warning')
            
            if mesh is None or mesh.n_points == 0:
                display(HTML("<p style='color:orange;'>⚠️ Could not load STL model.</p>"))
                return
            
            # Create plotter with notebook=True for Jupyter display
            plotter = pv.Plotter(notebook=True)
            
            # Add STL surface mesh with natural appearance
            plotter.add_mesh(
                mesh,
                show_edges=False,  # No edges for smooth surface
                color='lightgrey',  # Natural grey color
                opacity=0.7  # Slight transparency to see geometry
            )
            
            plotter.background_color = 'white'
            plotter.add_axes()
            plotter.add_text('STL Geometry (Data Generation Model)', font_size=12, position='upper_left')
            plotter.camera_position = 'iso'
            plotter.reset_camera()
            
            # Show in notebook
            plotter.show(jupyter_backend='static')
            display(HTML(f"<p>✅ STL geometry displayed ({mesh.n_points:,} vertices, {mesh.n_cells:,} faces)</p>"))
                    
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            log_message(f"STL visualization error: {str(e)}", 'error')
            
def visualize_hatching_with_pyslm():
    """Visualize hatching data using pyslm and PyVista."""
    try:
        import pyslm
        import pyvista as pv
        PYVISTA_AVAILABLE = True
    except ImportError:
        PYVISTA_AVAILABLE = False
        with hatching_output:
            clear_output(wait=True)
            display(HTML("<p style='color:red;'>❌ PyVista or pyslm not available. Install: pip install pyvista pyslm</p>"))
        return
    
    hatching_data = generated_data.get('hatching_layers')
    stl_model = generated_data.get('stl_model')
    
    if not hatching_data:
        with hatching_output:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No hatching data loaded. Generate or load data first.</p>"))
        return
    
    with hatching_output:
        clear_output(wait=True)
        display(HTML("<p>🔄 Generating hatching visualization...</p>"))
        
        try:
            plotter = pv.Plotter(notebook=True)
            
            # Add STL mesh if available
            if stl_model and isinstance(stl_model, dict):
                # Try to load STL file
                try:
                    from generation.process.stl_processor import STLProcessor
                    stl_processor = STLProcessor()
                    filename = stl_model.get('filename', '')
                    if filename:
                        stl_file_path = stl_processor.get_stl_file(filename)
                        if stl_file_path and stl_file_path.exists():
                            stl_part = pyslm.Part(stl_model.get('model_id', 'model'))
                            stl_part.setGeometry(str(stl_file_path))
                            stl_part.origin = [0.0, 0.0, 0.0]
                            stl_part.rotation = [0, 0, 0]
                            stl_part.dropToPlatform()
                            
                            # Convert to PyVista
                            if hasattr(stl_part, 'geometry') and stl_part.geometry is not None:
                                trimesh_mesh = stl_part.geometry
                                vertices = trimesh_mesh.vertices.copy()
                                faces = trimesh_mesh.faces
                                faces_pv = np.column_stack([np.full(len(faces), 3), faces]).flatten()
                                mesh = pv.PolyData(vertices, faces_pv)
                                plotter.add_mesh(mesh, color='lightgray', opacity=0.3, show_edges=False)
                except Exception as e:
                    log_message(f"Could not load STL: {str(e)}", 'warning')
            
            # Visualize hatching layers (sample a few layers)
            all_polylines = []
            sample_layers = hatching_data[:min(10, len(hatching_data))]  # Sample first 10 layers
            
            for layer_doc in sample_layers:
                z_pos = layer_doc.get('z_position', 0.0)
                # Note: Actual hatching geometry would need to be reconstructed from pyslm
                # This is a simplified visualization
                log_message(f"Visualizing layer at Z={z_pos:.2f} mm", 'info')
            
            plotter.add_text('Hatching Visualization (Sample Layers)', font_size=12)
            plotter.add_axes()
            plotter.show(jupyter_backend='static')
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_laser_parameters():
    """Visualize laser parameters with color mapping."""
    try:
        import pyvista as pv
        PYVISTA_AVAILABLE = True
    except ImportError:
        PYVISTA_AVAILABLE = False
        with laser_params_output:
            clear_output(wait=True)
            display(HTML("<p style='color:red;'>❌ PyVista not available. Install: pip install pyvista</p>"))
        return
    
    laser_data = generated_data.get('laser_parameters')
    
    if not laser_data:
        with laser_params_output:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No laser parameter data loaded. Generate or load data first.</p>"))
        return
    
    with laser_params_output:
        clear_output(wait=True)
        
        try:
            # Extract data
            points = np.array([[p['spatial_coordinates'][0], 
                              p['spatial_coordinates'][1], 
                              p['spatial_coordinates'][2]] for p in laser_data[:5000]])  # Limit for performance
            power = np.array([p['laser_power'] for p in laser_data[:5000]])
            speed = np.array([p['scan_speed'] for p in laser_data[:5000]])
            energy = np.array([p['energy_density'] for p in laser_data[:5000]])
            
            # Create PyVista point cloud
            point_cloud = pv.PolyData(points)
            point_cloud['power'] = power
            point_cloud['speed'] = speed
            point_cloud['energy'] = energy
            
            # Create plotter
            plotter = pv.Plotter(notebook=True)
            plotter.add_mesh(
                point_cloud,
                scalars='power',
                cmap='plasma',
                point_size=3,
                show_scalar_bar=True,
                scalar_bar_args={'title': 'Laser Power (W)'}
            )
            plotter.add_text('Laser Parameters Visualization', font_size=12)
            plotter.add_axes()
            plotter.show(jupyter_backend='static')
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_ispm_data():
    """Visualize ISPM data with 2D plots."""
    ispm_data = generated_data.get('ispm_data')
    
    if not ispm_data:
        with ispm_output:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No ISPM data loaded. Generate or load data first.</p>"))
        return
    
    with ispm_output:
        clear_output(wait=True)
        
        try:
            # Extract data
            temps = [p['melt_pool_temperature'] for p in ispm_data[:5000]]
            sizes = [p['melt_pool_size'] for p in ispm_data[:5000]]
            coords = np.array([[p['spatial_coordinates'][0], 
                               p['spatial_coordinates'][1]] for p in ispm_data[:5000]])
            
            # Create 2D plots
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # Temperature time series
            axes[0, 0].plot(temps[:1000], 'b-', alpha=0.7)
            axes[0, 0].set_xlabel('Sample Index')
            axes[0, 0].set_ylabel('Temperature (K)')
            axes[0, 0].set_title('Melt Pool Temperature Time Series')
            axes[0, 0].grid(True, alpha=0.3)
            
            # Melt pool size distribution
            axes[0, 1].hist(sizes, bins=50, color='green', alpha=0.7)
            axes[0, 1].set_xlabel('Melt Pool Size (mm)')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].set_title('Melt Pool Size Distribution')
            axes[0, 1].grid(True, alpha=0.3)
            
            # Spatial heatmap (XY projection)
            if len(coords) > 0:
                scatter = axes[1, 0].scatter(coords[:, 0], coords[:, 1], c=temps[:len(coords)], 
                                           cmap='hot', s=10, alpha=0.6)
                axes[1, 0].set_xlabel('X (mm)')
                axes[1, 0].set_ylabel('Y (mm)')
                axes[1, 0].set_title('Temperature Spatial Distribution (XY)')
                plt.colorbar(scatter, ax=axes[1, 0], label='Temperature (K)')
                axes[1, 0].grid(True, alpha=0.3)
            
            # Temperature vs Size scatter
            axes[1, 1].scatter(sizes, temps, alpha=0.5, s=10)
            axes[1, 1].set_xlabel('Melt Pool Size (mm)')
            axes[1, 1].set_ylabel('Temperature (K)')
            axes[1, 1].set_title('Temperature vs Melt Pool Size')
            axes[1, 1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_ct_scan():
    """Visualize CT scan data with 2D slice views."""
    ct_data = generated_data.get('ct_scan')
    
    if not ct_data:
        with ct_scan_output:
            clear_output(wait=True)
            display(HTML("<p style='color:orange;'>⚠️ No CT scan data loaded. Generate or load data first.</p>"))
        return
    
    with ct_scan_output:
        clear_output(wait=True)
        
        try:
            # Get density values
            density_values = ct_data.get('density_values')
            voxel_grid = ct_data.get('voxel_grid', {})
            
            if density_values is None:
                display(HTML("<p style='color:orange;'>⚠️ Density values not available. May need to load from GridFS.</p>"))
                return
            
            # Get dimensions
            dims = voxel_grid.get('dimensions', density_values.shape if hasattr(density_values, 'shape') else [100, 100, 100])
            
            # Create slice views
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # XY slice (middle Z)
            z_slice = dims[2] // 2 if isinstance(dims, list) else density_values.shape[2] // 2
            if isinstance(dims, list):
                im1 = axes[0, 0].imshow(density_values[:, :, z_slice], cmap='gray', origin='lower')
                axes[0, 0].set_title(f'XY Slice (Z = {z_slice})')
            else:
                im1 = axes[0, 0].imshow(density_values[:, :, z_slice], cmap='gray', origin='lower')
                axes[0, 0].set_title(f'XY Slice (Z = {z_slice})')
            plt.colorbar(im1, ax=axes[0, 0], label='Density')
            
            # XZ slice (middle Y)
            y_slice = dims[1] // 2 if isinstance(dims, list) else density_values.shape[1] // 2
            im2 = axes[0, 1].imshow(density_values[:, y_slice, :], cmap='gray', origin='lower')
            axes[0, 1].set_title(f'XZ Slice (Y = {y_slice})')
            plt.colorbar(im2, ax=axes[0, 1], label='Density')
            
            # YZ slice (middle X)
            x_slice = dims[0] // 2 if isinstance(dims, list) else density_values.shape[0] // 2
            im3 = axes[1, 0].imshow(density_values[x_slice, :, :], cmap='gray', origin='lower')
            axes[1, 0].set_title(f'YZ Slice (X = {x_slice})')
            plt.colorbar(im3, ax=axes[1, 0], label='Density')

            # Defect locations if available
            defects = ct_data.get('defect_locations', [])
            if defects and len(defects) > 0:
            # Handle both dict format {'x': ..., 'y': ..., 'z': ...} and list format [x, y, z]
                defect_coords_list = []
                for d in defects[:100]:  # Limit for display
                    if isinstance(d, dict):
                        # Dict format
                        x = d.get('x', 0)
                        y = d.get('y', 0)
                        z = d.get('z', 0)
                    elif isinstance(d, (list, tuple, np.ndarray)) and len(d) >= 3:
                        # List/tuple/array format [x, y, z]
                        x = d[0]
                        y = d[1]
                        z = d[2]
                    else:
                        # Skip invalid format
                        continue
                    defect_coords_list.append([x, y, z])
    
                if defect_coords_list:
                    defect_coords = np.array(defect_coords_list)
                    axes[1, 1].scatter(defect_coords[:, 0], defect_coords[:, 1], 
                              c=defect_coords[:, 2], cmap='Reds', s=50, alpha=0.7)
                    axes[1, 1].set_xlabel('X (mm)')
                    axes[1, 1].set_ylabel('Y (mm)')
                    axes[1, 1].set_title(f'Defect Locations ({len(defects)} total)')
                    axes[1, 1].grid(True, alpha=0.3)
                else:
                    axes[1, 1].text(0.5, 0.5, 'No valid defect coordinates', 
                           ha='center', va='center', transform=axes[1, 1].transAxes)
                    axes[1, 1].set_title('Defect Locations')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def on_preview_hatching_clicked(b):
    """Handle preview hatching button click."""
    visualize_hatching_with_pyslm()

def on_preview_laser_clicked(b):
    """Handle preview laser button click."""
    visualize_laser_parameters()

def on_preview_thermal_clicked(b):
    """Handle preview thermal/ISPM button click."""
    visualize_ispm_data()

def on_preview_ct_clicked(b):
    """Handle preview CT button click."""
    visualize_ct_scan()

# ============================================================================
# DATA LOADING AND VISUALIZATION FUNCTIONS
# ============================================================================

def refresh_model_dropdown():
    """Refresh the model dropdown with available models from MongoDB."""
    if not mongo_client or not mongo_client.is_connected():
        load_model_dropdown.options = ['Select a model to load...']
        return
    
    try:
        db = mongo_client.database
        models = list(db['stl_models'].find({}, {'model_id': 1, 'model_name': 1, 'filename': 1}).limit(100))
        
        options = ['Select a model to load...']
        for model in models:
            model_id = model.get('model_id', 'Unknown')
            model_name = model.get('model_name') or model.get('filename', 'Unknown')
            display_name = f"{model_name} ({model_id[:8]}...)"
            options.append((display_name, model_id))
        
        load_model_dropdown.options = options
        log_message(f"Found {len(models)} models in MongoDB", 'success')
    except Exception as e:
        log_message(f"Error refreshing model list: {str(e)}", 'error')
        load_model_dropdown.options = ['Select a model to load...']

def load_data_from_mongodb(model_id_input):
    """Load all data for a given model_id from MongoDB using UnifiedQueryClient."""
    # Handle tuple from dropdown: (display_name, model_id)
    if isinstance(model_id_input, tuple):
        model_id = model_id_input[1]  # Extract model_id from tuple
    else:
        model_id = str(model_id_input).strip()
    
    if not model_id or model_id == 'Select a model to load...':
        log_message("Invalid model_id", 'error')
        load_status_display.value = '<span style="color:red;">❌ Invalid model_id</span>'
        return None
    
    if not mongo_client or not mongo_client.is_connected():
        log_message("MongoDB not connected", 'error')
        load_status_display.value = '<span style="color:red;">❌ MongoDB not connected</span>'
        return None
    
    try:
        # Use UnifiedQueryClient like notebook 01
        from am_qadf.query import UnifiedQueryClient, SpatialQuery
        
        unified_client = UnifiedQueryClient(mongo_client=mongo_client)
        
        loaded_data = {
            'model_id': model_id,
            'stl_model': None,
            'hatching_layers': None,
            'laser_parameters': None,
            'ispm_data': None,
            'ct_scan': None
        }
        
        # Load STL model
        try:
            if unified_client.stl_client:
                stl_model = unified_client.stl_client.get_model(model_id)
                if stl_model:
                    loaded_data['stl_model'] = stl_model
                    log_message(f"Loaded STL model", 'success')
        except Exception as e:
            log_message(f"Error loading STL: {str(e)}", 'warning')
        
        # Load hatching layers
        try:
            if unified_client.hatching_client:
                hatching_layers = unified_client.hatching_client.get_layers(model_id)
                if hatching_layers:
                    loaded_data['hatching_layers'] = hatching_layers
                    layer_count = len(hatching_layers) if isinstance(hatching_layers, list) else 1
                    log_message(f"Loaded {layer_count} hatching layers", 'success')
        except Exception as e:
            log_message(f"Error loading hatching: {str(e)}", 'warning')
        
        # Load laser parameters
        try:
            if unified_client.laser_client:
                spatial_query = SpatialQuery(component_id=model_id)
                laser_result = unified_client.laser_client.query(spatial=spatial_query)
                if laser_result and hasattr(laser_result, 'points') and laser_result.points:
                    loaded_data['laser_parameters'] = laser_result
                    point_count = len(laser_result.points)
                    log_message(f"Loaded {point_count} laser parameter points", 'success')
        except Exception as e:
            log_message(f"Error loading laser parameters: {str(e)}", 'warning')
        
        # Load ISPM data
        try:
            if unified_client.ispm_client:
                spatial_query = SpatialQuery(component_id=model_id)
                ispm_result = unified_client.ispm_client.query(spatial=spatial_query)
                if ispm_result and hasattr(ispm_result, 'points') and ispm_result.points:
                    loaded_data['ispm_data'] = ispm_result
                    point_count = len(ispm_result.points)
                    log_message(f"Loaded {point_count} ISPM data points", 'success')
        except Exception as e:
            log_message(f"Error loading ISPM: {str(e)}", 'warning')
        
        # Load CT scan data
        try:
            if unified_client.ct_client:
                ct_result = unified_client.ct_client.get_scan(model_id)
                if ct_result:
                    loaded_data['ct_scan'] = ct_result
                    log_message("Loaded CT scan data", 'success')
        except Exception as e:
            log_message(f"Error loading CT scan: {str(e)}", 'warning')
        
        # Update generated_data with loaded data
        generated_data.update(loaded_data)
        
        # Calculate counts for status display
        stl_count = 1 if loaded_data['stl_model'] else 0
        hatching_count = len(loaded_data['hatching_layers']) if isinstance(loaded_data['hatching_layers'], list) else (1 if loaded_data['hatching_layers'] else 0)
        laser_count = len(loaded_data['laser_parameters'].points) if (loaded_data['laser_parameters'] and hasattr(loaded_data['laser_parameters'], 'points') and loaded_data['laser_parameters'].points) else 0
        ispm_count = len(loaded_data['ispm_data'].points) if (loaded_data['ispm_data'] and hasattr(loaded_data['ispm_data'], 'points') and loaded_data['ispm_data'].points) else 0
        ct_count = 1 if loaded_data['ct_scan'] else 0
        
        # Update status
        status_text = f'<b>✅ Loaded data for model: {model_id[:8]}...</b><br>'
        status_text += f'STL: {"✅" if stl_count > 0 else "❌"} | '
        status_text += f'Hatching: {hatching_count} layers | '
        status_text += f'Laser: {laser_count} points | '
        status_text += f'ISPM: {ispm_count} points | '
        status_text += f'CT: {"✅" if ct_count > 0 else "❌"}'
        load_status_display.value = status_text
        
        # Update visualization tabs to show data is ready
        with hatching_output:
            clear_output(wait=True)
            if hatching_count > 0:
                display(HTML(f"<p><b>✅ Data Ready for Visualization</b></p>"))
                display(HTML(f"<p>Hatching: {hatching_count} layers loaded</p>"))
                display(HTML(f"<p>STL: {'✅ Available' if stl_count > 0 else '❌ Not available'}</p>"))
                display(HTML(f"<p><i>Click 'Preview Hatching' button to visualize</i></p>"))
            else:
                display(HTML("<p><i>No hatching data available. Generate or load data first.</i></p>"))
        
        with laser_params_output:
            clear_output(wait=True)
            if laser_count > 0:
                display(HTML(f"<p><b>✅ Data Ready for Visualization</b></p>"))
                display(HTML(f"<p>Laser Parameters: {laser_count:,} points loaded</p>"))
                display(HTML(f"<p><i>Click 'Preview Laser Params' button to visualize</i></p>"))
            else:
                display(HTML("<p><i>No laser parameter data available. Generate or load data first.</i></p>"))
        
        with ispm_output:
            clear_output(wait=True)
            if ispm_count > 0:
                display(HTML(f"<p><b>✅ Data Ready for Visualization</b></p>"))
                display(HTML(f"<p>ISPM Data: {ispm_count:,} points loaded</p>"))
                display(HTML(f"<p><i>Click 'Preview ISPM Data' button to visualize</i></p>"))
            else:
                display(HTML("<p><i>No ISPM data available. Generate or load data first.</i></p>"))
        
        with ct_scan_output:
            clear_output(wait=True)
            if ct_count > 0:
                display(HTML(f"<p><b>✅ Data Ready for Visualization</b></p>"))
                display(HTML(f"<p>CT Scan: Data loaded</p>"))
                display(HTML(f"<p><i>Click 'Preview CT Scan' button to visualize</i></p>"))
            else:
                display(HTML("<p><i>No CT scan data available. Generate or load data first.</i></p>"))
        
        log_message(f"Data loading complete for model {model_id[:8]}...", 'success')
        return loaded_data
        
    except Exception as e:
        log_message(f"Error loading data: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        load_status_display.value = f'<span style="color:red;">❌ Error: {str(e)}</span>'
        return None

def on_load_model_clicked(b):
    """Handle load model button click."""
    model_id_input = load_model_dropdown.value
    if model_id_input == 'Select a model to load...':
        log_message("Please select a model from the dropdown", 'warning')
        return
    
    # Handle tuple from dropdown
    if isinstance(model_id_input, tuple):
        model_id = model_id_input[1]
    else:
        model_id = str(model_id_input)
    
    log_message(f"Loading data for model: {model_id[:8]}...", 'info')
    load_data_from_mongodb(model_id_input)

def on_load_by_id_clicked(b):
    """Handle load by ID button click."""
    model_id = load_model_id_input.value.strip()
    if not model_id:
        log_message("Please enter a model_id", 'warning')
        return
    
    log_message(f"Loading data for model_id: {model_id[:8]}...", 'info')
    load_data_from_mongodb(model_id)

def visualize_hatching_with_pyslm():
    """Visualize hatching data using custom HatchingVisualizer (inspired by pyslm)."""
    with hatching_output:
        clear_output(wait=True)
        
        hatching_data = generated_data.get('hatching_layers')
        stl_model = generated_data.get('stl_model')
        
        if not hatching_data or len(hatching_data) == 0:
            display(HTML("<p style='color:orange;'>⚠️ No hatching data loaded. Generate or load data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating hatching visualization...</p>"))
        
        try:
            from am_qadf.visualization.hatching_visualizer import HatchingVisualizer
            import matplotlib.pyplot as plt
            
            visualizer = HatchingVisualizer()
            
            # Use MongoDB data directly (reconstructed)
            sample_layers = hatching_data[:min(20, len(hatching_data))]  # Sample first 10 layers
            
            fig, ax = visualizer.plot_layers(
                layers=sample_layers,  # MongoDB documents
                plot_contours=True,
                plot_hatches=True,
                plot_points=False,
                plot_3d=True,
                plot_colorbar=True,  # Single colorbar for all layers
                index='laser_power',  # Color by laser power (from MongoDB)
                colormap='plasma',
                linewidth=0.5
            )
            
            ax.set_title(f'Hatching Visualization ({len(sample_layers)} layers)')
            ax.set_xlabel('X (mm)')
            ax.set_ylabel('Y (mm)')
            ax.set_zlabel('Z (mm)')
            
            plt.tight_layout()
            plt.show()
            
            display(HTML(f"<p>✅ Visualized {len(sample_layers)} layers</p>"))
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            log_message(f"Visualization error: {str(e)}", 'error')
            

def visualize_laser_parameters():
    """Visualize laser parameters with color mapping."""
    with laser_params_output:
        clear_output(wait=True)
        
        try:
            import pyvista as pv
            PYVISTA_AVAILABLE = True
        except ImportError:
            display(HTML("<p style='color:red;'>❌ PyVista not available. Install: pip install pyvista</p>"))
            return
        
        laser_data = generated_data.get('laser_parameters')
        
        if not laser_data:
            display(HTML("<p style='color:orange;'>⚠️ No laser parameter data loaded. Generate or load data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating laser parameters visualization...</p>"))
        
        try:
            # Handle QueryResult object (from UnifiedQueryClient)
            from am_qadf.query.base_query_client import QueryResult
            if isinstance(laser_data, QueryResult):
                if not laser_data.points or len(laser_data.points) == 0:
                    display(HTML("<p style='color:orange;'>⚠️ No laser parameter points in QueryResult</p>"))
                    return
                
                # Extract points (limit for performance)
                max_points = 10000
                points_array = np.array(laser_data.points)
                if len(points_array) > max_points:
                    # Random sample for better performance
                    indices = np.random.choice(len(points_array), max_points, replace=False)
                    points = points_array[indices]
                else:
                    points = points_array
                
                # Extract signals
                signals = laser_data.signals or {}
                
                # Get signal arrays (they should match points length)
                if 'laser_power' in signals:
                    power_array = np.array(signals['laser_power'])
                    if len(points) <= max_points:
                        power = power_array
                    else:
                        power = power_array[indices]
                else:
                    power = np.ones(len(points)) * 250.0  # Default
                
                if 'scan_speed' in signals:
                    speed_array = np.array(signals['scan_speed'])
                    if len(points) <= max_points:
                        speed = speed_array
                    else:
                        speed = speed_array[indices]
                else:
                    speed = np.ones(len(points)) * 500.0  # Default
                
                if 'energy_density' in signals:
                    energy_array = np.array(signals['energy_density'])
                    if len(points) <= max_points:
                        energy = energy_array
                    else:
                        energy = energy_array[indices]
                else:
                    # Calculate from power and speed if available
                    energy = power / (speed + 1e-10)  # Avoid division by zero
            else:
                # Handle list of dicts (from direct MongoDB query)
                points = np.array([[p['spatial_coordinates'][0], 
                                  p['spatial_coordinates'][1], 
                                  p['spatial_coordinates'][2]] for p in laser_data[:5000]])
                power = np.array([p['laser_power'] for p in laser_data[:5000]])
                speed = np.array([p['scan_speed'] for p in laser_data[:5000]])
                energy = np.array([p['energy_density'] for p in laser_data[:5000]])
            
            # Ensure arrays are correct length and same type
            min_len = min(len(points), len(power), len(speed), len(energy))
            points = np.array(points[:min_len])
            power = np.array(power[:min_len]) if isinstance(power, np.ndarray) else np.array(power[:min_len])
            speed = np.array(speed[:min_len]) if isinstance(speed, np.ndarray) else np.array(speed[:min_len])
            energy = np.array(energy[:min_len]) if isinstance(energy, np.ndarray) else np.array(energy[:min_len])
            
            # Ensure points are 3D
            if points.shape[1] == 2:
                # Add Z coordinate if missing
                z_coords = np.zeros((points.shape[0], 1))
                points = np.hstack([points, z_coords])
            
            # Create PyVista point cloud
            point_cloud = pv.PolyData(points)
            point_cloud['power'] = power
            point_cloud['speed'] = speed
            point_cloud['energy'] = energy
            
            # Create plotter
            plotter = pv.Plotter(notebook=True)
            plotter.add_mesh(
                point_cloud,
                scalars='power',
                cmap='plasma',
                point_size=5,
                show_scalar_bar=True,
                scalar_bar_args={'title': 'Laser Power (W)', 'n_labels': 5}
            )
            plotter.add_text(f'Laser Parameters Visualization ({len(points):,} points)', font_size=12)
            plotter.add_axes()
            plotter.show(jupyter_backend='static')
            
            log_message(f"Visualized {len(points):,} laser parameter points", 'success')
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_ispm_data():
    """Visualize ISPM data with 2D plots."""
    with ispm_output:
        clear_output(wait=True)
        
        ispm_data = generated_data.get('ispm_data')
        
        if not ispm_data:
            display(HTML("<p style='color:orange;'>⚠️ No ISPM data loaded. Generate or load data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating ISPM data visualization...</p>"))
        
        try:
            # Handle QueryResult object (from UnifiedQueryClient)
            from am_qadf.query.base_query_client import QueryResult
            if isinstance(ispm_data, QueryResult):
                if not ispm_data.points or len(ispm_data.points) == 0:
                    display(HTML("<p style='color:orange;'>⚠️ No ISPM points in QueryResult</p>"))
                    return
                
                # Limit points for visualization
                max_points = 5000
                points_array = np.array(ispm_data.points)
                if len(points_array) > max_points:
                    indices = np.random.choice(len(points_array), max_points, replace=False)
                    points = points_array[indices]
                else:
                    points = points_array
                    indices = np.arange(len(points))
                
                signals = ispm_data.signals or {}
                
                # Extract signal arrays
                if 'melt_pool_temperature' in signals:
                    temps_array = np.array(signals['melt_pool_temperature'])
                    temps = temps_array[indices] if len(temps_array) > max_points else temps_array
                else:
                    temps = np.ones(len(points)) * 1700.0  # Default
                
                if 'melt_pool_size' in signals:
                    sizes_array = np.array(signals['melt_pool_size'])
                    sizes = sizes_array[indices] if len(sizes_array) > max_points else sizes_array
                else:
                    sizes = np.ones(len(points)) * 0.5  # Default
                
                # Extract coordinates (XY for 2D plots, Z for info)
                if points.shape[1] >= 2:
                    coords = points[:, :2]
                    if points.shape[1] >= 3:
                        z_coords = points[:, 2]
                        z_min = float(np.min(z_coords))
                        z_max = float(np.max(z_coords))
                        z_mean = float(np.mean(z_coords))
                        z_info = f"Z: {z_min:.2f} to {z_max:.2f} mm (mean: {z_mean:.2f} mm)"
                    else:
                        z_info = "Z: N/A (2D data)"
                else:
                    coords = np.array([[p[0], p[1] if len(p) > 1 else 0] for p in points])
                    z_info = "Z: N/A"
            else:
                # Handle list of dicts (from direct MongoDB query)
                temps = [p['melt_pool_temperature'] for p in ispm_data[:5000]]
                sizes = [p['melt_pool_size'] for p in ispm_data[:5000]]
                coords = np.array([[p['spatial_coordinates'][0], 
                                   p['spatial_coordinates'][1]] for p in ispm_data[:5000]])
                
                # Extract Z coordinates
                z_coords_list = []
                for p in ispm_data[:5000]:
                    if 'spatial_coordinates' in p and len(p['spatial_coordinates']) >= 3:
                        z_coords_list.append(p['spatial_coordinates'][2])
                
                if z_coords_list:
                    z_min = min(z_coords_list)
                    z_max = max(z_coords_list)
                    z_mean = sum(z_coords_list) / len(z_coords_list)
                    z_info = f"Z: {z_min:.2f} to {z_max:.2f} mm (mean: {z_mean:.2f} mm)"
                else:
                    z_info = "Z: N/A"
            
            # Create 2D plots
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # Temperature time series
            axes[0, 0].plot(temps[:1000], 'b-', alpha=0.7)
            axes[0, 0].set_xlabel('Sample Index')
            axes[0, 0].set_ylabel('Temperature (K)')
            axes[0, 0].set_title('Melt Pool Temperature Time Series')
            axes[0, 0].grid(True, alpha=0.3)
            
            # Melt pool size distribution
            axes[0, 1].hist(sizes, bins=50, color='green', alpha=0.7)
            axes[0, 1].set_xlabel('Melt Pool Size (mm)')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].set_title('Melt Pool Size Distribution')
            axes[0, 1].grid(True, alpha=0.3)
            
            # Spatial heatmap (XY projection) with Z info
            if len(coords) > 0:
                scatter = axes[1, 0].scatter(coords[:, 0], coords[:, 1], c=temps[:len(coords)], 
                                           cmap='hot', s=10, alpha=0.6)
                axes[1, 0].set_xlabel('X (mm)')
                axes[1, 0].set_ylabel('Y (mm)')
                axes[1, 0].set_title(f'Temperature Spatial Distribution (XY Projection)\n{z_info}')
                plt.colorbar(scatter, ax=axes[1, 0], label='Temperature (K)')
                axes[1, 0].grid(True, alpha=0.3)
            else:
                axes[1, 0].text(0.5, 0.5, 'No coordinate data', 
                               ha='center', va='center', transform=axes[1, 0].transAxes)
                axes[1, 0].set_title('Temperature Spatial Distribution')
            
            # Temperature vs Size scatter
            axes[1, 1].scatter(sizes, temps, alpha=0.5, s=10)
            axes[1, 1].set_xlabel('Melt Pool Size (mm)')
            axes[1, 1].set_ylabel('Temperature (K)')
            axes[1, 1].set_title('Temperature vs Melt Pool Size')
            axes[1, 1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            
def visualize_ct_scan():
    """Visualize CT scan data with 2D slice views."""
    with ct_scan_output:
        clear_output(wait=True)
        
        ct_data = generated_data.get('ct_scan')
        
        if not ct_data:
            display(HTML("<p style='color:orange;'>⚠️ No CT scan data loaded. Generate or load data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating CT scan visualization...</p>"))
        
        try:
            # Handle QueryResult or dict from get_scan()
            # Note: get_scan() returns a dict, not QueryResult
            from am_qadf.query.base_query_client import QueryResult
            
            # get_scan() returns a dict, so handle it as dict
            if isinstance(ct_data, dict):
                voxel_grid = ct_data.get('voxel_grid', {})
                defects = ct_data.get('defect_locations', [])
                
                # Try to get density_values from dict first
                density_values = ct_data.get('density_values')
                
                # If not in dict, try to load from GridFS using CT client
                if density_values is None and mongo_client and mongo_client.is_connected():
                    try:
                        from am_qadf.query import CTScanClient
                        ct_client = CTScanClient(mongo_client=mongo_client, use_mongodb=True)
                        model_id = generated_data.get('model_id')
                        if model_id:
                            density_values = ct_client.get_density_values(model_id)
                            if density_values is not None:
                                log_message("Loaded density values from GridFS", 'success')
                    except Exception as e:
                        log_message(f"Could not load density from GridFS: {str(e)}", 'warning')
            elif isinstance(ct_data, QueryResult):
                # QueryResult (unlikely but handle it)
                if hasattr(ct_data, 'signals') and ct_data.signals:
                    density_values = ct_data.signals.get('density_values')
                elif hasattr(ct_data, 'metadata'):
                    density_values = ct_data.metadata.get('density_values')
                else:
                    density_values = None
                voxel_grid = ct_data.metadata.get('voxel_grid', {}) if hasattr(ct_data, 'metadata') else {}
                defects = ct_data.metadata.get('defect_locations', []) if (hasattr(ct_data, 'metadata') and ct_data.metadata) else []
            else:
                display(HTML(f"<p style='color:orange;'>⚠️ Unexpected CT data type: {type(ct_data)}</p>"))
                return
            
            if density_values is None:
                display(HTML("<p style='color:orange;'>⚠️ Density values not available.</p>"))
                display(HTML("<p>CT scan metadata loaded, but density array not found in GridFS.</p>"))
                if defects and len(defects) > 0:
                    display(HTML(f"<p>Defect locations available: {len(defects)} defects</p>"))
                return
            
            # Get dimensions
            dims = voxel_grid.get('dimensions', density_values.shape if hasattr(density_values, 'shape') else [100, 100, 100])
            
            # Create slice views
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            # XY slice (middle Z)
            z_slice = dims[2] // 2 if isinstance(dims, list) else density_values.shape[2] // 2
            if isinstance(dims, list):
                im1 = axes[0, 0].imshow(density_values[:, :, z_slice], cmap='gray', origin='lower')
                axes[0, 0].set_title(f'XY Slice (Z = {z_slice})')
            else:
                im1 = axes[0, 0].imshow(density_values[:, :, z_slice], cmap='gray', origin='lower')
                axes[0, 0].set_title(f'XY Slice (Z = {z_slice})')
            plt.colorbar(im1, ax=axes[0, 0], label='Density')
            
            # XZ slice (middle Y)
            y_slice = dims[1] // 2 if isinstance(dims, list) else density_values.shape[1] // 2
            im2 = axes[0, 1].imshow(density_values[:, y_slice, :], cmap='gray', origin='lower')
            axes[0, 1].set_title(f'XZ Slice (Y = {y_slice})')
            plt.colorbar(im2, ax=axes[0, 1], label='Density')
            
            # YZ slice (middle X)
            x_slice = dims[0] // 2 if isinstance(dims, list) else density_values.shape[0] // 2
            im3 = axes[1, 0].imshow(density_values[x_slice, :, :], cmap='gray', origin='lower')
            axes[1, 0].set_title(f'YZ Slice (X = {x_slice})')
            plt.colorbar(im3, ax=axes[1, 0], label='Density')
            
            # Defect locations if available
            defects = ct_data.get('defect_locations', [])
            if defects and len(defects) > 0:
                # defect_coords = np.array([[d.get('x', 0), d.get('y', 0), d.get('z', 0)] 
                #                          for d in defects[:100]])  # Limit for display
                # # Handle both dict format {'x': ..., 'y': ..., 'z': ...} and list format [x, y, z]
                defect_coords_list = []
                for d in defects[:100]:  # Limit for display
                    if isinstance(d, dict):
                        # Dict format
                        x = d.get('x', 0)
                        y = d.get('y', 0)
                        z = d.get('z', 0)
                    elif isinstance(d, (list, tuple)) and len(d) >= 3:
                        # List/tuple format [x, y, z]
                        x = d[0]
                        y = d[1]
                        z = d[2]
                    else:
                        # Skip invalid format
                        continue
                    defect_coords_list.append([x, y, z])

            if defect_coords_list:
                defect_coords = np.array(defect_coords_list)
                axes[1, 1].scatter(defect_coords[:, 0], defect_coords[:, 1], 
                                  c=defect_coords[:, 2], cmap='Reds', s=50, alpha=0.7)
                axes[1, 1].set_xlabel('X (mm)')
                axes[1, 1].set_ylabel('Y (mm)')
                axes[1, 1].set_title(f'Defect Locations ({len(defects)} total)')
                axes[1, 1].grid(True, alpha=0.3)
            else:
                axes[1, 1].text(0.5, 0.5, 'No defect data', 
                               ha='center', va='center', transform=axes[1, 1].transAxes)
                axes[1, 1].set_title('Defect Locations')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

# def on_preview_hatching_clicked(b):
#     """Handle preview hatching button click."""
#     visualize_hatching_with_pyslm()

def on_preview_stl_clicked(b):
    """Handle preview hatching button click."""
    with stl_output:
        clear_output(wait=True)
        display(HTML("<p>🔍 Button clicked! Testing output widget...</p>"))
        print("Button handler called successfully")
    visualize_stl_voxel_image()
    
def on_preview_hatching_clicked(b):
    """Handle preview hatching button click."""
    with hatching_output:
        clear_output(wait=True)
        display(HTML("<p>🔍 Button clicked! Testing output widget...</p>"))
        print("Button handler called successfully")
    visualize_hatching_with_pyslm()


def on_preview_laser_clicked(b):
    """Handle preview laser button click."""
    visualize_laser_parameters()

def on_preview_thermal_clicked(b):
    """Handle preview thermal/ISPM button click."""
    visualize_ispm_data()

def on_preview_ct_clicked(b):
    """Handle preview CT button click."""
    visualize_ct_scan()


# Attach Event Handlers
stl_load_button.on_click(on_stl_load_clicked)
preview_button.on_click(on_preview_clicked)
generate_button.on_click(on_generate_clicked)
clear_preview_button.on_click(on_clear_preview_clicked)
save_config_button.on_click(on_save_config_clicked)
refresh_mongo_button.on_click(on_refresh_mongo_clicked)
stl_selector.observe(on_stl_selector_change, names='value')
collections_to_populate.observe(on_collections_change, names='value')

# Add new event handlers
load_model_button.on_click(on_load_model_clicked)
load_by_id_button.on_click(on_load_by_id_clicked)
preview_hatching_button.on_click(on_preview_hatching_clicked)
preview_laser_button.on_click(on_preview_laser_clicked)
preview_thermal_button.on_click(on_preview_thermal_clicked)
preview_ct_button.on_click(on_preview_ct_clicked)


# Initialize
update_generate_button_state()
on_refresh_mongo_clicked(None)
refresh_model_dropdown()  # Add this line


# Display the complete interface
display(complete_interface)
print("✅ Interface created with event handlers - ready to use!")


✅ Interface created with event handlers - ready to use!
